<a href="https://colab.research.google.com/github/Xaomomo/Unet-with-attention/blob/main/LSH_attention_Test_versions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1st attempt at LSH Attention using trax in conjunction with TF keras

Primero se usara trax para intentar usar el concepto de LSH self attention, ya que esta libreria fue diseñada por google, los que propusieron el transformer en el paper del reformer.

In [ ]:
!gdown "1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t
To: /content/input.zip
100% 367M/367M [00:01<00:00, 250MB/s]


In [ ]:
!pip install -q -U trax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 67.7 MB/s eta 0:00:00


In [ ]:
!pip install -q -U git+https://github.com/google/trax@master

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.9 MB/s eta 0:00:00


In [ ]:
!unzip -q input.zip

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 1

In [ ]:
import nibabel as nib
import os
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import cv2
import torch

In [ ]:
import imageio as iio
import glob
from skimage.transform import resize
import PIL
from PIL import Image
src="/content/input/train"
imag="/images/"
X=np.zeros((len(glob.glob(src+imag+"*.png")),IMG_WIDTH,IMG_HEIGHT,1))
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x!="/content/input/train/images/coronacases_009_149.png":
    im=Image.open(x)
    X[i,:,:,0]=im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST)
mas="/Amasks/"
Y=np.zeros((len(glob.glob(src+mas+"*.png")),IMG_WIDTH,IMG_HEIGHT))
for i,x in enumerate(sorted(glob.glob(src+mas+"*.png"))):
  if x!="/content/input/train/Amasks/coronacases_009_149.png":
    im=Image.open(x)
    im=np.array(im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST))
    im[im==85]=1
    im[im==170]=2
    im[im==255]=3
    Y[i,:,:]=im
import tensorflow as tf
print(Y.shape)
print(np.unique(Y))
Y=tf.keras.utils.to_categorical(Y)
print(Y.shape)
print(np.unique(Y))

(3520, 128, 128)
[0. 1. 2. 3.]
(3520, 128, 128, 4)
[0. 1.]


In [ ]:
src="/content/input/train"
imag="/images/"
Siz=0
id=0
ri=0
metadat=[]
c="001"
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x.find("radiopaedia") == -1:
    x=x[x.find("_")+1:]
    x=x[:x.find("_")]
  else:
    x=x[x.find("_")+1:]
    x=x[:x.find("_",9)]
  if c!=x:
    print(x)
    metadat.append([id,Siz,ri])
    ri=i
    c=x
    id+=1
    Siz=0
  Siz+=1
metadat.append([id,Siz,ri])

002
003
004
005
006
007
008
009
010
10_85902_1
10_85902_3
14_85914_0
27_86410_0
29_86490_1
29_86491_1
36_86526_0
40_86625_0
4_85506_1
7_85703_0


##New Split

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=1, shuffle=True)
IDX=[x for x in range(10)]
IDY=[x for x in range(10)]
for train_index, val_index in cv.split(metadat,metadat):
  print(train_index, val_index)

[ 0  1  2  4  5  7  8  9 11 12 13 14 15 17 18 19] [ 3  6 10 16]
[ 0  1  3  5  6  7  8  9 10 11 12 13 15 16 18 19] [ 2  4 14 17]
[ 2  3  4  5  6  8  9 10 11 12 14 15 16 17 18 19] [ 0  1  7 13]
[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 16 17] [ 9 15 18 19]
[ 0  1  2  3  4  6  7  9 10 13 14 15 16 17 18 19] [ 5  8 11 12]


In [ ]:
for i,x in enumerate(train_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_train=X[ri:ri+size]
    Y_train=Y[ri:ri+size]
  else:
    X_train=np.concatenate((X_train,X[ri:ri+size]),axis=0)
    Y_train=np.concatenate((Y_train,Y[ri:ri+size]),axis=0)
for i,x in enumerate(val_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_test=X[ri:ri+size]
    Y_test=Y[ri:ri+size]
  else:
    X_test=np.concatenate((X_test,X[ri:ri+size]),axis=0)
    Y_test=np.concatenate((Y_test,Y[ri:ri+size]),axis=0)

##Patch wise

/usr/local/lib/python3.9/dist-packages/trax/layers/research/efficient_attention.py

In [ ]:
from keras.layers import Layer
import keras.backend as K
import torch.nn as nn
import tensorflow as tf
import trax

In [ ]:
def DiceMetric(y_true, y_pred):
  smooth=1e-6
  gama=2
  y_true, y_pred = tf.cast(
      y_true, dtype=tf.float32), tf.cast(y_pred, tf.float32)
  nominator = 2 * \
      tf.reduce_sum(tf.multiply(y_pred, y_true)) + smooth
  denominator = tf.reduce_sum(
      y_pred ** gama) + tf.reduce_sum(y_true ** gama) + smooth
  result = tf.divide(nominator, denominator)
  return result
def DiceLoss(y_true, y_pred):
      result= 1- DiceMetric(y_true, y_pred)
      return result

In [ ]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou
def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

In [ ]:
def dice_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      #None,128,128,1
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      #None,128,128
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += dice_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def iou_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += iou_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def conv_block(X,f,d=0.1,group=1):
  c = tf.keras.layers.Conv2D(f[0], (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',groups=group)(X)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[1], (3, 3), kernel_initializer='he_normal', padding='same', groups=group)(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  s = tf.keras.layers.Conv2D(f[1], (1, 1), kernel_initializer='he_normal', padding='same')(X)
  s = tf.keras.layers.BatchNormalization(axis=3)(s)
  c = tf.keras.layers.Add()([c,s])
  c = tf.keras.layers.ReLU()(c)
  return c,c

In [ ]:
import sys

In [ ]:
def Flatten(M):
  s1=M.shape[1]
  s2=M.shape[2]
  s3=M.shape[3]
  flat=s1*s2
  M= tf.reshape(M,[-1,flat,s3])
  return M,s1,s2,s3

In [ ]:
trax.fastmath.set_backend('tensorflow-numpy')

In [ ]:
trax.fastmath.set_backend('numpy')

In [ ]:
def Flatten_LSH(M,N,nheads,dimk,k=128):
  M,s1,s2,s3= Flatten(M)
  N,_,_,_= Flatten(N)
  layer=trax.layers.PureLSHSelfAttention(n_heads=nheads,d_qk=dimk,d_v=dimk,mode="train")
  #dimk=s3
  #print(M.dtype.as_numpy_dtype)
  #layer.weights=(tf.random.uniform(shape=[dimk,dimk]),tf.random.uniform(shape=[dimk,dimk]),tf.random.uniform(shape=[dimk,dimk]))
  layer=trax.AsKeras(layer,batch_size=16,dtype=tf.float64)
  pm=layer((M,M))
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
def vit(X,Y,n_heads=16,psize=16,k=256,dim="4D"):
  npatch=int(X.shape[1]/psize)
  psizey=int(psize/2)
  kdim=int(X.shape[3]/n_heads)
  Y=tf.keras.layers.Dense(X.shape[3])(Y)
  if npatch==0:
    pm=Flatten_LSH(X,Y,n_heads,kdim,k)
    return pm
  for i in range(npatch):
    st=i*psize
    sty=i*psizey
    end=st+psize
    endy=sty+psizey
    for j in range(npatch):
      st2=j*psize
      end2=st2+psize
      sty2=j*psizey
      endy2=sty2+psizey
      patch= X[:,st:end,st2:end2,:]
      patch2= Y[:,sty:endy,sty2:endy2,:]
      pm=Flatten_LSH(patch,patch2,n_heads,kdim,k)
      if j==0:
        A=pm
      else:
        A=tf.concat([A,pm],2)
    if i==0:
      V=A
    else:
      V=tf.concat([V,A],1)
  return V



### Vit

In [ ]:
import tensorflow as tf

def create_model():
  nheads=4
  sizep=32
  gr=4
  kval=128
  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path

  m1= vit(z4,z5,nheads,sizep,k=kval)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= vit(z3,z6,nheads,sizep,k=kval)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= vit(z2,z7,nheads,sizep,k=kval)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= vit(z1,z8,nheads,sizep,k=kval)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###LSH without patches

In [ ]:
def Linformer_lay(M,nheads,k=128):
  s1=M.shape[1]
  s2=M.shape[2]
  s3=M.shape[3]
  flat=s1*s2
  M= tf.reshape(M,[-1,flat,s3])
  kdim=int(s3/nheads)
  layer=trax.layers.LSHSelfAttention(n_heads=nheads,d_qk=kdim,d_v=kdim,mode="train")
  layer=trax.AsKeras(layer,batch_size=16,dtype=tf.float64)
  pm=layer((M,M))
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
import tensorflow as tf

def create_model():
  nheads=16
  sizep=32
  gr=4

  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path

  m1= Linformer_lay(z4,nheads)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= Linformer_lay(z3,nheads)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= Linformer_lay(z2,nheads)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= Linformer_lay(z1,nheads)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###Test

In [ ]:
model=create_model()
model.fit(X_train,Y_train,batch_size=16,epochs=25)

StagingError: ignored

In [ ]:
scores= model.evaluate(X_test, Y_test, verbose=0)
print(f'Score for fold {0}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')

Score for fold 0: loss of 0.02161642350256443; accuracy of 99.28983449935913% iou_coef_multilabel of 75.75774192810059% dice_coef_multilabel of 79.42734956741333%


Patch size=32 k=2:

Score for fold 0: loss of 0.19930902123451233; accuracy of 94.81756091117859% iou_coef_multilabel of 67.01422333717346% dice_coef_multilabel of 75.7577657699585%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

####Patch test 5D

k=2

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.018923498690128326; accuracy of 99.33900833129883% iou_coef_multilabel of 76.09699368476868% dice_coef_multilabel of 79.55656051635742%
Score for fold 2: loss of 0.02973838523030281; accuracy of 99.06825423240662% iou_coef_multilabel of 76.13354325294495% dice_coef_multilabel of 80.5853009223938%
Score for fold 3: loss of 0.03030165657401085; accuracy of 99.11888837814331% iou_coef_multilabel of 75.47945976257324% dice_coef_multilabel of 79.76462244987488%
Score for fold 4: loss of 0.027371248230338097; accuracy of 99.03544187545776% iou_coef_multilabel of 74.86760020256042% dice_coef_multilabel of 80.22578358650208%
Score for fold 5: loss of 0.024255475029349327; accuracy of 99.22391176223755% iou_coef_multilabel of 75.09616017341614% dice_coef_multilabel of 78.89301180839539%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01604907028377056; accuracy of 99.4849443435669% iou_coef_multilabel of 82.25744366645813% dice_coef_multilabel of 85.06359457969666%
Score for fold 2: loss of 0.035502493381500244; accuracy of 98.98368120193481% iou_coef_multilabel of 81.15251660346985% dice_coef_multilabel of 85.50392389297485%
Score for fold 3: loss of 0.03146236762404442; accuracy of 99.13598895072937% iou_coef_multilabel of 80.20720481872559% dice_coef_multilabel of 84.49628949165344%
Score for fold 4: loss of 0.029628699645400047; accuracy of 98.99090528488159% iou_coef_multilabel of 80.18351197242737% dice_coef_multilabel of 85.33194661140442%
Score for fold 5: loss of 0.025924744084477425; accuracy of 99.26274418830872% iou_coef_multilabel of 79.02063727378845% dice_coef_multilabel of 82.51100778579712%


test 1 25 epochs:

Este test se diferencia de los siguientes porque la primera capa del decoder, tenia una falla en el skip connection, como lo estaba estructurando este toma key y query del output de la capa del nivel y la del anterior, pero sin darme cuenta deje esa, como dos del mismo nivel.

Score for fold 1: loss of 0.01725444197654724; accuracy of 99.44809079170227% iou_coef_multilabel of 76.75567865371704% dice_coef_multilabel of 79.90655899047852%

Score for fold 2: loss of 0.026257488876581192; accuracy of 99.15213584899902% iou_coef_multilabel of 77.25991010665894% dice_coef_multilabel of 81.667959690094%

Score for fold 3: loss of 0.029813755303621292; accuracy of 99.0189254283905% iou_coef_multilabel of 73.47208261489868% dice_coef_multilabel of 77.98171043395996%

Score for fold 4: loss of 0.028865722939372063; accuracy of 98.88771772384644% iou_coef_multilabel of 73.73278737068176% dice_coef_multilabel of 79.40745949745178%

Score for fold 5: loss of 0.02384837530553341; accuracy of 99.24745559692383% iou_coef_multilabel of 72.86607623100281% dice_coef_multilabel of 76.55618190765381%

test 1 50 epochs:

Score for fold 1: loss of 0.01484227366745472; accuracy of 99.52776432037354% iou_coef_multilabel of 83.22951197624207% dice_coef_multilabel of 85.81125140190125%

Score for fold 2: loss of 0.03430720046162605; accuracy of 99.037766456604% iou_coef_multilabel of 81.77401423454285% dice_coef_multilabel of 86.07376217842102%

Score for fold 3: loss of 0.02642207406461239; accuracy of 99.15657639503479% iou_coef_multilabel of 79.4792652130127% dice_coef_multilabel of 83.80299210548401%

Score for fold 4: loss of 0.03244418650865555; accuracy of 98.78917336463928% iou_coef_multilabel of 76.53632164001465% dice_coef_multilabel of 82.26258158683777%

Score for fold 5: loss of 0.02530766651034355; accuracy of 99.25082325935364% iou_coef_multilabel of 77.88779139518738% dice_coef_multilabel of 81.41918778419495%

test 2 25 epochs:
Desde aqui estan los kfolds con las capas correctamente implementadas.

Score for fold 1: loss of 0.015737926587462425; accuracy of 99.46310520172119% iou_coef_multilabel of 78.279709815979% dice_coef_multilabel of 81.3614547252655%

Score for fold 2: loss of 0.03170934319496155; accuracy of 98.95466566085815% iou_coef_multilabel of 75.30025839805603% dice_coef_multilabel of 79.98074293136597%

Score for fold 3: loss of 0.027374574914574623; accuracy of 99.1090476512909% iou_coef_multilabel of 75.07091164588928% dice_coef_multilabel of 79.41552400588989%

Score for fold 4: loss of 0.03620793670415878; accuracy of 98.70395064353943% iou_coef_multilabel of 72.66994714736938% dice_coef_multilabel of 78.63662838935852%

Score for fold 5: loss of 0.02868451550602913; accuracy of 99.15916919708252% iou_coef_multilabel of 77.30659246444702% dice_coef_multilabel of 80.87429404258728%

test 2 50 epochs:

Score for fold 1: loss of 0.017423078417778015; accuracy of 99.4379997253418% iou_coef_multilabel of 77.40354537963867% dice_coef_multilabel of 80.43342232704163%

Score for fold 2: loss of 0.038378071039915085; accuracy of 98.94863367080688% iou_coef_multilabel of 79.24267053604126% dice_coef_multilabel of 83.60227346420288%

Score for fold 3: loss of 0.028568308800458908; accuracy of 99.16971921920776% iou_coef_multilabel of 80.07370233535767% dice_coef_multilabel of 84.15423035621643%

Score for fold 4: loss of 0.03010973334312439; accuracy of 99.00146722793579% iou_coef_multilabel of 80.8301031589508% dice_coef_multilabel of 85.89409589767456%

Score for fold 5: loss of 0.02312619984149933; accuracy of 99.28799271583557% iou_coef_multilabel of 79.67525124549866% dice_coef_multilabel of 83.12292695045471%


k=256

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.016601325944066048; accuracy of 99.49246644973755% iou_coef_multilabel of 80.1440417766571% dice_coef_multilabel of 82.72783160209656%
Score for fold 2: loss of 0.02792888507246971; accuracy of 99.11463856697083% iou_coef_multilabel of 77.49691009521484% dice_coef_multilabel of 81.86010122299194%
Score for fold 3: loss of 0.033090461045503616; accuracy of 99.0357756614685% iou_coef_multilabel of 75.54389238357544% dice_coef_multilabel of 79.85571622848511%
Score for fold 4: loss of 0.02735254541039467; accuracy of 99.02375340461731% iou_coef_multilabel of 75.38833618164062% dice_coef_multilabel of 80.59991002082825%
Score for fold 5: loss of 0.02401789091527462; accuracy of 99.26612973213196% iou_coef_multilabel of 76.66497826576233% dice_coef_multilabel of 80.13312816619873%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01717947982251644; accuracy of 99.4581937789917% iou_coef_multilabel of 79.65168952941895% dice_coef_multilabel of 82.61964917182922%
Score for fold 2: loss of 0.03299105167388916; accuracy of 99.06346797943115% iou_coef_multilabel of 78.3797025680542% dice_coef_multilabel of 82.73652791976929%
Score for fold 3: loss of 0.034237224608659744; accuracy of 99.15516972541809% iou_coef_multilabel of 78.44327688217163% dice_coef_multilabel of 82.67260193824768%


KeyboardInterrupt: ignored

#Second attempt at lsh attention, using the trax library

El problema que encontre en el uso de LSH Attention usando trax, es que despues de que logre que funcionara, este fue horrible en termino de tiempo de procesamiento. Una epoch duraba mas que un fold completo en los demas modelos, por lo que se decidio usar esta libreria, esperando que el problema de lo lento que era el modelo es solo culpa de AsKeras, que era la forma en que se traducia la capa de trax a keras.

In [ ]:
! pip install -q -U trax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.9 MB/s eta 0:00:00


In [ ]:
import trax
import numpy as np
import trax.layers as tl
from trax.fastmath import numpy as jnp

In [ ]:
def Softmax():
  r"""Returns a layer that computes the softmax function.
  .. math::
      f(x) = e^x\sum(e^x)
  """
  return tl.base.Fn('Softmax', lambda x: jnp.exp(x)/sum(jnp.exp(x)))

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()  # upload kaggle.json
! mkdir ~/.kaggle

Saving kaggle.json to kaggle.json


In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                                title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
zusmani/pakistan-toshakhana-files                                  Pakistan ToshaKhana Files                             1MB  2023-03-15 16:25:43           2293         95  1.0              
ramkrijal/tomato-daily-prices                                      Tomato Daily Prices                                  10KB  2023-03-10 15:39:14           1220         37  1.0              
themrityunjaypathak/covid-cases-and-deaths-worldwide               Covid Cases and Deaths WorldWide                      8KB  2023-02-01 12:22:51          14181        467  1.0              
datascientistanna/customers-dataset          

In [ ]:
! kaggle datasets download -d dansbecker/cityscapes-image-pairs

100% 201M/202M [00:07<00:00, 29.4MB/s]
100% 202M/202M [00:07<00:00, 26.6MB/s]


In [ ]:
competition_name = 'carvana-image-masking-challenge'

In [ ]:
! unzip -q cityscapes-image-pairs.zip

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar100_n',
 'cifar10_1',
 'cifar10_corrupted',
 'cifar10_n',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'conll2002',
 'conll2003',
 'controlled_noisy_web_labels'

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incompleteWKTUVQ/oxford_iiit_pet-train.tfrecord*...:…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incompleteWKTUVQ/oxford_iiit_pet-test.tfrecord*...: …

Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask
def load_image(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(
    datapoint['segmentation_mask'],
    (128, 128),
    method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
  )

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
import tensorflow as tf

In [ ]:
trax.fastmath.set_backend('numpy')

In [ ]:
trax.fastmath.set_backend('tensorflow-numpy')

In [ ]:
print(info)

tfds.core.DatasetInfo(
    name='oxford_iiit_pet',
    full_name='oxford_iiit_pet/3.2.0',
    description="""
    The Oxford-IIIT pet dataset is a 37 category pet image dataset with roughly 200
    images for each class. The images have large variations in scale, pose and
    lighting. All images have an associated ground truth annotation of breed.
    """,
    homepage='http://www.robots.ox.ac.uk/~vgg/data/pets/',
    data_path='/root/tensorflow_datasets/oxford_iiit_pet/3.2.0',
    file_format=tfrecord,
    download_size=773.52 MiB,
    dataset_size=774.69 MiB,
    features=FeaturesDict({
        'file_name': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=37),
        'segmentation_mask': Image(shape=(None, None, 1), dtype=uint8),
        'species': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    sp

In [ ]:
# let's fix batch size
batch_size = 32

In [ ]:
from trax import data

inputs = data.inputs.Serial(
  data.TFDS('oxford_iiit_pet:3.*.*', keys=('image', 'segmentation_mask'), train=True),
  data.Shuffle(),
  trax.data.Batch(batch_size),
  lambda g: map(lambda p: (p[0].astype(np.float32), jnp.squeeze(p[1],axis=3)), g)
)
val=data.inputs.Serial(
  data.tf_inputs.TFDS('oxford_iiit_pet:3.*.*', keys=('image', 'segmentation_mask'), train=False),
  trax.data.Batch(batch_size),
  lambda g: map(lambda p: (p[0].astype(np.float32), jnp.squeeze(p[1],axis=3)), g)
)

/usr/local/lib/python3.9/dist-packages/jax/_src/xla_bridge.py:613: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(


In [ ]:
type(inputs)

function

In [ ]:
import os
import os.path as osp
from PIL import Image
from itertools import cycle
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def color_kmean(root):
  """ creates a k-means objects that recognizes all 13 colors of dataset. """

  # take 10 first images
  files = os.listdir(root)[:10]
  colors = list()
  for f in files:
    img = load_image(osp.join(root, f))
    # total width
    w = img.shape[2]
    # get the right half of image, which is the label image
    img = img[:, w:, :]
    # collect all the colors present in label image
    colors.append(img.reshape(-1, 3))

  colors = np.array(colors)
  colors = colors.reshape(-1, 3)

  # finally, fit all the colors into the KMeans
  kmeans = KMeans(13)
  kmeans.fit(colors)

  return kmeans

def load_image(path):
  """ loading an image. """

  assert osp.exists(path), path + " not found"
  image = Image.open(path)
  image = np.asarray(image)
  return image

def color2class(segs, km):
  """
  given an label image, convert it to class matrix,
  which is a 2D matrix of class labels (scalars).
  """

  h, w, c = segs.shape
  segs = segs.reshape((-1, 3))
  segs = km.predict(segs)
  segs = segs.reshape((h, w, 1))
  return segs

def load_dataset(root, km):
  """ load dataset. """
  index = 0
  imgs_path = [osp.join(root, f) for f in os.listdir(root)]

  # load images one by one, finally, and image and
  # its label matrix is returned
  while True:
    img = load_image(imgs_path[index])
    w = img.shape[1] // 2
    img, seg = img[:, :w, :], img[:, w:, :]

    seg = color2class(seg, km)

    seg = seg.reshape(-1)
    assert img.shape == (256, 256, 3), img.shape
    assert seg.shape == (256 * 256,), seg.shape
    yield img, seg

    index = (index + 1) % len(imgs_path)

In [ ]:
root = 'cityscapes_data'

trainset_path = osp.join(root, 'train')
valset_path = osp.join(root, 'val')

km = color_kmean(trainset_path)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
train_dataset = load_dataset(trainset_path, km)
val_dataset = load_dataset(valset_path, km)
print(type(train_dataset))
train_transforms = trax.data.Serial(
    trax.data.Shuffle(),
    trax.data.Batch(batch_size),
    lambda g: map(lambda p: (p[0].astype(np.float32), p[1]), g),
)
val_transforms = trax.data.Serial(
    trax.data.Batch(batch_size),
    lambda g: map(lambda p: (p[0].astype(np.float32), p[1]), g),
)

train_dataset = train_transforms(train_dataset)
val_dataset = val_transforms(val_dataset)

NameError: ignored

In [ ]:
type(train_dataset)

map

In [ ]:
def CrossEntropy3d(criterion_2d):
  """ returns 3D cross entropy loss function """
  def _loss_fn(output, target):
    output = output.reshape(-1, 1)
    target = target.reshape(-1,)
    loss = criterion_2d((output, target))
    return loss
  return _loss_fn

# check dataset
x, y = next(train_dataset)
print(x.shape, y.shape)
print(x.dtype, y.dtype)

NameError: ignored

In [ ]:
x, y = next(inputs)
print(x.shape, y.shape)
print(x.dtype, y.dtype)

TypeError: ignored

In [ ]:
# set learning rate
lr = 1e-2

# create new trax Fn for new loss fn, and provide it a name
#criterion = trax.layers.base.Fn("CrossEntropy3d",
#                                CrossEntropy3d(tl.CategoryCrossEntropy())                                )
criterion = tl.CrossEntropyLoss()
# create TrainTask
train_task = trax.supervised.training.TrainTask(
    labeled_data=inputs(),
    loss_layer=criterion,
    optimizer=trax.optimizers.Adam(),
    n_steps_per_checkpoint=50
)

# create EvalTask
eval_task = trax.supervised.training.EvalTask(
    labeled_data=val(),
    metrics=[criterion]
)

In [ ]:
def ConvBlock(kernel_size, filters, norm, non_linearity=tl.Relu,
              mode='train'):
  """ResNet convolutional striding block."""
  ks = kernel_size
  filters1, filters2, filters3 = filters
  main = [
      tl.Conv(filters1, (1, 1),padding='SAME'),
      norm(mode=mode),
      non_linearity(),
      tl.Conv(filters2, (ks, ks), padding='SAME'),
      norm(mode=mode),
      non_linearity(),
      tl.Conv(filters3, (1, 1),padding='SAME'),
      norm(mode=mode),
  ]
  shortcut = [
      tl.Conv(filters3, (1, 1)),
      norm(mode=mode),
  ]
  return [
      tl.Residual(main, shortcut=shortcut),
      non_linearity()
  ]

In [ ]:
def ConvBlock(kernel_size, filters, norm, non_linearity=tl.Relu,
              mode='train'):
  filters1, filters2, filters3 = filters
  return tl.Serial(
      tl.Conv(filters1, (2, 2),padding='SAME'),
      non_linearity(),
      tl.Conv(filters1, (2, 2),padding='SAME'),
      non_linearity(),
  )

In [ ]:
def Unet(kernel_size, filter, norm=tl.BatchNorm, non_linearity=tl.Relu,
              mode='train',steps=5):
  f1=filter[steps-1]
  f2=filter[steps-1]
  f3=filter[steps-1]
  filters=[f1,f2,f3]
  if steps==1:
    return [tl.pooling.MaxPool((2,2),padding="SAME"),
            ConvBlock(kernel_size, filters, norm, non_linearity,
              mode='train')]
  shortcut=None
  return tl.Serial(tl.pooling.MaxPool((2,2),padding="SAME"),
                   ConvBlock(kernel_size, filters, norm, non_linearity,mode='train'),
                   tl.combinators.Branch(Unet(kernel_size, filter, norm, non_linearity,mode='train',steps=steps-1),shortcut),
                   tl.deconvolution.ConvTranspose(filter[steps-1],(2,2),padding="SAME"),
                   tl.combinators.Concatenate(n_items=2,axis=3),
                   ConvBlock(kernel_size, filters, norm, non_linearity,mode='train'))


In [ ]:
model = tl.Serial(
    Unet(3,[256,128,64,32,16],mode="train",steps=4),
    tl.Conv(1, (1, 1)),
    tl.core.Softmax()
)

In [ ]:
training_loop = trax.supervised.training.Loop(
    model,
    train_task,
    eval_tasks=[eval_task],
    output_dir=None
)

training_loop.run(500)

LayerError: ignored

#3rd attempt at LSH Attention using full tensorflow

El problema fue que trax no funcionaba eficientemente en terminos de ram, considerando la dificultad del modelo.

In [ ]:
!gdown "1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t
To: /content/input.zip
100% 367M/367M [00:01<00:00, 203MB/s]


In [ ]:
!git clone https://github.com/cerebroai/reformers.git

Cloning into 'reformers'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 82 (delta 37), reused 81 (delta 36), pack-reused 0
Unpacking objects: 100% (82/82), 288.38 KiB | 1.72 MiB/s, done.


In [ ]:
!unzip -q input.zip

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 1

In [ ]:
import nibabel as nib
import os
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import cv2
import torch

In [ ]:
import imageio as iio
import glob
from skimage.transform import resize
import PIL
from PIL import Image
src="/content/input/train"
imag="/images/"
X=np.zeros((len(glob.glob(src+imag+"*.png")),IMG_WIDTH,IMG_HEIGHT,1))
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x!="/content/input/train/images/coronacases_009_149.png":
    im=Image.open(x)
    X[i,:,:,0]=im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST)
mas="/Amasks/"
Y=np.zeros((len(glob.glob(src+mas+"*.png")),IMG_WIDTH,IMG_HEIGHT))
for i,x in enumerate(sorted(glob.glob(src+mas+"*.png"))):
  if x!="/content/input/train/Amasks/coronacases_009_149.png":
    im=Image.open(x)
    im=np.array(im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST))
    im[im==85]=1
    im[im==170]=2
    im[im==255]=3
    Y[i,:,:]=im
import tensorflow as tf
print(Y.shape)
print(np.unique(Y))
Y=tf.keras.utils.to_categorical(Y)
print(Y.shape)
print(np.unique(Y))

(3520, 128, 128)
[0. 1. 2. 3.]
(3520, 128, 128, 4)
[0. 1.]


In [ ]:
src="/content/input/train"
imag="/images/"
Siz=0
id=0
ri=0
metadat=[]
c="001"
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x.find("radiopaedia") == -1:
    x=x[x.find("_")+1:]
    x=x[:x.find("_")]
  else:
    x=x[x.find("_")+1:]
    x=x[:x.find("_",9)]
  if c!=x:
    print(x)
    metadat.append([id,Siz,ri])
    ri=i
    c=x
    id+=1
    Siz=0
  Siz+=1
metadat.append([id,Siz,ri])

002
003
004
005
006
007
008
009
010
10_85902_1
10_85902_3
14_85914_0
27_86410_0
29_86490_1
29_86491_1
36_86526_0
40_86625_0
4_85506_1
7_85703_0


##New Split

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=1, shuffle=True)
IDX=[x for x in range(10)]
IDY=[x for x in range(10)]
for train_index, val_index in cv.split(metadat,metadat):
  print(train_index, val_index)

[ 0  1  2  4  5  7  8  9 11 12 13 14 15 17 18 19] [ 3  6 10 16]
[ 0  1  3  5  6  7  8  9 10 11 12 13 15 16 18 19] [ 2  4 14 17]
[ 2  3  4  5  6  8  9 10 11 12 14 15 16 17 18 19] [ 0  1  7 13]
[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 16 17] [ 9 15 18 19]
[ 0  1  2  3  4  6  7  9 10 13 14 15 16 17 18 19] [ 5  8 11 12]


In [ ]:
for i,x in enumerate(train_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_train=X[ri:ri+size]
    Y_train=Y[ri:ri+size]
  else:
    X_train=np.concatenate((X_train,X[ri:ri+size]),axis=0)
    Y_train=np.concatenate((Y_train,Y[ri:ri+size]),axis=0)
for i,x in enumerate(val_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_test=X[ri:ri+size]
    Y_test=Y[ri:ri+size]
  else:
    X_test=np.concatenate((X_test,X[ri:ri+size]),axis=0)
    Y_test=np.concatenate((Y_test,Y[ri:ri+size]),axis=0)

##Patch wise

/content/reformers/reformers/TFefficient_attention.py

/content/reformers/reformers/TFutils.py

In [ ]:
from keras.layers import Layer
import keras.backend as K
import torch.nn as nn
import tensorflow as tf
from importlib import reload
from reformers.reformers.TFefficient_attention import TFLSHAttention,TFLSHSelfAttention

In [ ]:
def DiceMetric(y_true, y_pred):
  smooth=1e-6
  gama=2
  y_true, y_pred = tf.cast(
      y_true, dtype=tf.float32), tf.cast(y_pred, tf.float32)
  nominator = 2 * \
      tf.reduce_sum(tf.multiply(y_pred, y_true)) + smooth
  denominator = tf.reduce_sum(
      y_pred ** gama) + tf.reduce_sum(y_true ** gama) + smooth
  result = tf.divide(nominator, denominator)
  return result
def DiceLoss(y_true, y_pred):
      result= 1- DiceMetric(y_true, y_pred)
      return result

In [ ]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou
def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

In [ ]:
def dice_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      #None,128,128,1
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      #None,128,128
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += dice_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def iou_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += iou_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def conv_block(X,f,d=0.1,group=1):
  c = tf.keras.layers.Conv2D(f[0], (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',groups=group)(X)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[1], (3, 3), kernel_initializer='he_normal', padding='same', groups=group)(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  s = tf.keras.layers.Conv2D(f[1], (1, 1), kernel_initializer='he_normal', padding='same')(X)
  s = tf.keras.layers.BatchNormalization(axis=3)(s)
  c = tf.keras.layers.Add()([c,s])
  c = tf.keras.layers.ReLU()(c)
  return c,c

In [ ]:
import sys

In [ ]:
def Flatten(M):
  shapes=M.shape
  s1=shapes[1]
  s2=shapes[2]
  s3=shapes[3]
  flat=s1*s2
  #M= tf.reshape(M,shape=[-1,-1,s3])
  M=tf.reshape(M,shape=[-1, s1 * s2 , s3])
  return M,s1,s2,s3

In [ ]:
def Flatten_LSH(M,N,nheads,dimk,k=128):
  M,s1,s2,s3= Flatten(M)
  N,_,_,_= Flatten(N)
  pm=TFLSHAttention(dropout=0.2)(M,M)
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
def Flatten_LSH(M,N,nheads,dimk,k=128):
  M,s1,s2,s3= Flatten(M)
  N,_,_,_= Flatten(N)
  pm=TFLSHSelfAttention(s3,heads=1,dropout=0.2,bucket_size=8,n_hashes=1)(M)
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
def vit(X,Y,n_heads=16,psize=16,k=256,dim="4D"):
  npatch=int(X.shape[1]/psize)
  psizey=int(psize/2)
  kdim=int(X.shape[3]/n_heads)
  Y=tf.keras.layers.Dense(X.shape[3])(Y)
  if npatch==0:
    pm=Flatten_LSH(X,Y,n_heads,kdim,k)
    return pm
  for i in range(npatch):
    st=i*psize
    sty=i*psizey
    end=st+psize
    endy=sty+psizey
    for j in range(npatch):
      st2=j*psize
      end2=st2+psize
      sty2=j*psizey
      endy2=sty2+psizey
      patch= X[:,st:end,st2:end2,:]
      patch2= Y[:,sty:endy,sty2:endy2,:]
      pm=Flatten_LSH(patch,patch2,n_heads,kdim,k)
      if j==0:
        A=pm
      else:
        A=tf.concat([A,pm],2)
    if i==0:
      V=A
    else:
      V=tf.concat([V,A],1)
  return V



### Vit

In [ ]:
import tensorflow as tf

def create_model():
  nheads=4
  sizep=32
  gr=4
  kval=128
  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),batch_size=16)
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path

  m1= vit(z4,z5,nheads,sizep,k=kval)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= vit(z3,z6,nheads,sizep,k=kval)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= vit(z2,z7,nheads,sizep,k=kval)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= vit(z1,z8,nheads,sizep,k=kval)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###LSH without patches

In [ ]:
def Linformer_lay(M,nheads,k=128):
  s1=M.shape[1]
  s2=M.shape[2]
  s3=M.shape[3]
  flat=s1*s2
  M= tf.reshape(M,[-1,flat,s3])
  kdim=int(s3/nheads)
  layer=trax.layers.LSHSelfAttention(n_heads=nheads,d_qk=kdim,d_v=kdim,mode="train")
  layer=trax.AsKeras(layer,batch_size=16,dtype=tf.float64)
  pm=layer((M,M))
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
import tensorflow as tf

def create_model():
  nheads=16
  sizep=32
  gr=4

  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path

  m1= Linformer_lay(z4,nheads)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= Linformer_lay(z3,nheads)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= Linformer_lay(z2,nheads)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= Linformer_lay(z1,nheads)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###Test

In [ ]:
model=create_model()
model.fit(X_train,Y_train,batch_size=16,epochs=25)

ValueError: ignored

In [ ]:
scores= model.evaluate(X_test, Y_test, verbose=0)
print(f'Score for fold {0}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')

Score for fold 0: loss of 0.02161642350256443; accuracy of 99.28983449935913% iou_coef_multilabel of 75.75774192810059% dice_coef_multilabel of 79.42734956741333%


Patch size=32 k=2:

Score for fold 0: loss of 0.19930902123451233; accuracy of 94.81756091117859% iou_coef_multilabel of 67.01422333717346% dice_coef_multilabel of 75.7577657699585%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

####Patch test 5D

k=2

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.018923498690128326; accuracy of 99.33900833129883% iou_coef_multilabel of 76.09699368476868% dice_coef_multilabel of 79.55656051635742%
Score for fold 2: loss of 0.02973838523030281; accuracy of 99.06825423240662% iou_coef_multilabel of 76.13354325294495% dice_coef_multilabel of 80.5853009223938%
Score for fold 3: loss of 0.03030165657401085; accuracy of 99.11888837814331% iou_coef_multilabel of 75.47945976257324% dice_coef_multilabel of 79.76462244987488%
Score for fold 4: loss of 0.027371248230338097; accuracy of 99.03544187545776% iou_coef_multilabel of 74.86760020256042% dice_coef_multilabel of 80.22578358650208%
Score for fold 5: loss of 0.024255475029349327; accuracy of 99.22391176223755% iou_coef_multilabel of 75.09616017341614% dice_coef_multilabel of 78.89301180839539%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01604907028377056; accuracy of 99.4849443435669% iou_coef_multilabel of 82.25744366645813% dice_coef_multilabel of 85.06359457969666%
Score for fold 2: loss of 0.035502493381500244; accuracy of 98.98368120193481% iou_coef_multilabel of 81.15251660346985% dice_coef_multilabel of 85.50392389297485%
Score for fold 3: loss of 0.03146236762404442; accuracy of 99.13598895072937% iou_coef_multilabel of 80.20720481872559% dice_coef_multilabel of 84.49628949165344%
Score for fold 4: loss of 0.029628699645400047; accuracy of 98.99090528488159% iou_coef_multilabel of 80.18351197242737% dice_coef_multilabel of 85.33194661140442%
Score for fold 5: loss of 0.025924744084477425; accuracy of 99.26274418830872% iou_coef_multilabel of 79.02063727378845% dice_coef_multilabel of 82.51100778579712%


test 1 25 epochs:

Este test se diferencia de los siguientes porque la primera capa del decoder, tenia una falla en el skip connection, como lo estaba estructurando este toma key y query del output de la capa del nivel y la del anterior, pero sin darme cuenta deje esa, como dos del mismo nivel.

Score for fold 1: loss of 0.01725444197654724; accuracy of 99.44809079170227% iou_coef_multilabel of 76.75567865371704% dice_coef_multilabel of 79.90655899047852%

Score for fold 2: loss of 0.026257488876581192; accuracy of 99.15213584899902% iou_coef_multilabel of 77.25991010665894% dice_coef_multilabel of 81.667959690094%

Score for fold 3: loss of 0.029813755303621292; accuracy of 99.0189254283905% iou_coef_multilabel of 73.47208261489868% dice_coef_multilabel of 77.98171043395996%

Score for fold 4: loss of 0.028865722939372063; accuracy of 98.88771772384644% iou_coef_multilabel of 73.73278737068176% dice_coef_multilabel of 79.40745949745178%

Score for fold 5: loss of 0.02384837530553341; accuracy of 99.24745559692383% iou_coef_multilabel of 72.86607623100281% dice_coef_multilabel of 76.55618190765381%

test 1 50 epochs:

Score for fold 1: loss of 0.01484227366745472; accuracy of 99.52776432037354% iou_coef_multilabel of 83.22951197624207% dice_coef_multilabel of 85.81125140190125%

Score for fold 2: loss of 0.03430720046162605; accuracy of 99.037766456604% iou_coef_multilabel of 81.77401423454285% dice_coef_multilabel of 86.07376217842102%

Score for fold 3: loss of 0.02642207406461239; accuracy of 99.15657639503479% iou_coef_multilabel of 79.4792652130127% dice_coef_multilabel of 83.80299210548401%

Score for fold 4: loss of 0.03244418650865555; accuracy of 98.78917336463928% iou_coef_multilabel of 76.53632164001465% dice_coef_multilabel of 82.26258158683777%

Score for fold 5: loss of 0.02530766651034355; accuracy of 99.25082325935364% iou_coef_multilabel of 77.88779139518738% dice_coef_multilabel of 81.41918778419495%

test 2 25 epochs:
Desde aqui estan los kfolds con las capas correctamente implementadas.

Score for fold 1: loss of 0.015737926587462425; accuracy of 99.46310520172119% iou_coef_multilabel of 78.279709815979% dice_coef_multilabel of 81.3614547252655%

Score for fold 2: loss of 0.03170934319496155; accuracy of 98.95466566085815% iou_coef_multilabel of 75.30025839805603% dice_coef_multilabel of 79.98074293136597%

Score for fold 3: loss of 0.027374574914574623; accuracy of 99.1090476512909% iou_coef_multilabel of 75.07091164588928% dice_coef_multilabel of 79.41552400588989%

Score for fold 4: loss of 0.03620793670415878; accuracy of 98.70395064353943% iou_coef_multilabel of 72.66994714736938% dice_coef_multilabel of 78.63662838935852%

Score for fold 5: loss of 0.02868451550602913; accuracy of 99.15916919708252% iou_coef_multilabel of 77.30659246444702% dice_coef_multilabel of 80.87429404258728%

test 2 50 epochs:

Score for fold 1: loss of 0.017423078417778015; accuracy of 99.4379997253418% iou_coef_multilabel of 77.40354537963867% dice_coef_multilabel of 80.43342232704163%

Score for fold 2: loss of 0.038378071039915085; accuracy of 98.94863367080688% iou_coef_multilabel of 79.24267053604126% dice_coef_multilabel of 83.60227346420288%

Score for fold 3: loss of 0.028568308800458908; accuracy of 99.16971921920776% iou_coef_multilabel of 80.07370233535767% dice_coef_multilabel of 84.15423035621643%

Score for fold 4: loss of 0.03010973334312439; accuracy of 99.00146722793579% iou_coef_multilabel of 80.8301031589508% dice_coef_multilabel of 85.89409589767456%

Score for fold 5: loss of 0.02312619984149933; accuracy of 99.28799271583557% iou_coef_multilabel of 79.67525124549866% dice_coef_multilabel of 83.12292695045471%


k=256

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.016601325944066048; accuracy of 99.49246644973755% iou_coef_multilabel of 80.1440417766571% dice_coef_multilabel of 82.72783160209656%
Score for fold 2: loss of 0.02792888507246971; accuracy of 99.11463856697083% iou_coef_multilabel of 77.49691009521484% dice_coef_multilabel of 81.86010122299194%
Score for fold 3: loss of 0.033090461045503616; accuracy of 99.0357756614685% iou_coef_multilabel of 75.54389238357544% dice_coef_multilabel of 79.85571622848511%
Score for fold 4: loss of 0.02735254541039467; accuracy of 99.02375340461731% iou_coef_multilabel of 75.38833618164062% dice_coef_multilabel of 80.59991002082825%
Score for fold 5: loss of 0.02401789091527462; accuracy of 99.26612973213196% iou_coef_multilabel of 76.66497826576233% dice_coef_multilabel of 80.13312816619873%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01717947982251644; accuracy of 99.4581937789917% iou_coef_multilabel of 79.65168952941895% dice_coef_multilabel of 82.61964917182922%
Score for fold 2: loss of 0.03299105167388916; accuracy of 99.06346797943115% iou_coef_multilabel of 78.3797025680542% dice_coef_multilabel of 82.73652791976929%
Score for fold 3: loss of 0.034237224608659744; accuracy of 99.15516972541809% iou_coef_multilabel of 78.44327688217163% dice_coef_multilabel of 82.67260193824768%


KeyboardInterrupt: ignored

#LSH with pytorch

In [ ]:
!gdown "1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t
To: /content/input.zip
100% 367M/367M [00:06<00:00, 60.4MB/s]


In [ ]:
!git clone https://github.com/cerebroai/reformers.git

Cloning into 'reformers'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 82 (delta 37), reused 81 (delta 36), pack-reused 0
Unpacking objects: 100% (82/82), 288.38 KiB | 1007.00 KiB/s, done.


In [ ]:
!unzip -q input.zip

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 1

In [ ]:
import nibabel as nib
import os
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import cv2
import torch

In [ ]:
import imageio as iio
import glob
from skimage.transform import resize
import PIL
from PIL import Image
src="/content/input/train"
imag="/images/"
X=np.zeros((len(glob.glob(src+imag+"*.png")),IMG_WIDTH,IMG_HEIGHT,1))
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x!="/content/input/train/images/coronacases_009_149.png":
    im=Image.open(x)
    X[i,:,:,0]=im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST)
mas="/Amasks/"
Y=np.zeros((len(glob.glob(src+mas+"*.png")),IMG_WIDTH,IMG_HEIGHT))
for i,x in enumerate(sorted(glob.glob(src+mas+"*.png"))):
  if x!="/content/input/train/Amasks/coronacases_009_149.png":
    im=Image.open(x)
    im=np.array(im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST))
    im[im==85]=1
    im[im==170]=2
    im[im==255]=3
    Y[i,:,:]=im
import tensorflow as tf
print(Y.shape)
print(np.unique(Y))
Y=tf.keras.utils.to_categorical(Y)
print(Y.shape)
print(np.unique(Y))

(3520, 128, 128)
[0. 1. 2. 3.]
(3520, 128, 128, 4)
[0. 1.]


In [ ]:
src="/content/input/train"
imag="/images/"
Siz=0
id=0
ri=0
metadat=[]
c="001"
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x.find("radiopaedia") == -1:
    x=x[x.find("_")+1:]
    x=x[:x.find("_")]
  else:
    x=x[x.find("_")+1:]
    x=x[:x.find("_",9)]
  if c!=x:
    print(x)
    metadat.append([id,Siz,ri])
    ri=i
    c=x
    id+=1
    Siz=0
  Siz+=1
metadat.append([id,Siz,ri])

002
003
004
005
006
007
008
009
010
10_85902_1
10_85902_3
14_85914_0
27_86410_0
29_86490_1
29_86491_1
36_86526_0
40_86625_0
4_85506_1
7_85703_0


##New Split

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=1, shuffle=True)
IDX=[x for x in range(10)]
IDY=[x for x in range(10)]
for train_index, val_index in cv.split(metadat,metadat):
  print(train_index, val_index)

[ 0  1  2  4  5  7  8  9 11 12 13 14 15 17 18 19] [ 3  6 10 16]
[ 0  1  3  5  6  7  8  9 10 11 12 13 15 16 18 19] [ 2  4 14 17]
[ 2  3  4  5  6  8  9 10 11 12 14 15 16 17 18 19] [ 0  1  7 13]
[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 16 17] [ 9 15 18 19]
[ 0  1  2  3  4  6  7  9 10 13 14 15 16 17 18 19] [ 5  8 11 12]


In [ ]:
for i,x in enumerate(train_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_train=X[ri:ri+size]
    Y_train=Y[ri:ri+size]
  else:
    X_train=np.concatenate((X_train,X[ri:ri+size]),axis=0)
    Y_train=np.concatenate((Y_train,Y[ri:ri+size]),axis=0)
for i,x in enumerate(val_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_test=X[ri:ri+size]
    Y_test=Y[ri:ri+size]
  else:
    X_test=np.concatenate((X_test,X[ri:ri+size]),axis=0)
    Y_test=np.concatenate((Y_test,Y[ri:ri+size]),axis=0)

## Unet Base

In [ ]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

#LSH attention with no attention between chunks as keras multihead subclass

In [ ]:
!gdown "1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t
To: /content/input.zip
100% 367M/367M [00:04<00:00, 74.1MB/s]


In [ ]:
!unzip -q input.zip

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 1

In [ ]:
import nibabel as nib
import os
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import cv2
import torch

In [ ]:
import imageio as iio
import glob
from skimage.transform import resize
import PIL
from PIL import Image
src="/content/input/train"
imag="/images/"
X=np.zeros((len(glob.glob(src+imag+"*.png")),IMG_WIDTH,IMG_HEIGHT,1))
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x!="/content/input/train/images/coronacases_009_149.png":
    im=Image.open(x)
    X[i,:,:,0]=im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST)
mas="/Amasks/"
Y=np.zeros((len(glob.glob(src+mas+"*.png")),IMG_WIDTH,IMG_HEIGHT))
for i,x in enumerate(sorted(glob.glob(src+mas+"*.png"))):
  if x!="/content/input/train/Amasks/coronacases_009_149.png":
    im=Image.open(x)
    im=np.array(im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST))
    im[im==85]=1
    im[im==170]=2
    im[im==255]=3
    Y[i,:,:]=im
import tensorflow as tf
print(Y.shape)
print(np.unique(Y))
Y=tf.keras.utils.to_categorical(Y)
print(Y.shape)
print(np.unique(Y))

(3520, 128, 128)
[0. 1. 2. 3.]
(3520, 128, 128, 4)
[0. 1.]


In [ ]:
src="/content/input/train"
imag="/images/"
Siz=0
id=0
ri=0
metadat=[]
c="001"
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x.find("radiopaedia") == -1:
    x=x[x.find("_")+1:]
    x=x[:x.find("_")]
  else:
    x=x[x.find("_")+1:]
    x=x[:x.find("_",9)]
  if c!=x:
    print(x)
    metadat.append([id,Siz,ri])
    ri=i
    c=x
    id+=1
    Siz=0
  Siz+=1
metadat.append([id,Siz,ri])

002
003
004
005
006
007
008
009
010
10_85902_1
10_85902_3
14_85914_0
27_86410_0
29_86490_1
29_86491_1
36_86526_0
40_86625_0
4_85506_1
7_85703_0


##New Split

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=1, shuffle=True)
IDX=[x for x in range(10)]
IDY=[x for x in range(10)]
for train_index, val_index in cv.split(metadat,metadat):
  print(train_index, val_index)

[ 0  1  2  4  5  7  8  9 11 12 13 14 15 17 18 19] [ 3  6 10 16]
[ 0  1  3  5  6  7  8  9 10 11 12 13 15 16 18 19] [ 2  4 14 17]
[ 2  3  4  5  6  8  9 10 11 12 14 15 16 17 18 19] [ 0  1  7 13]
[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 16 17] [ 9 15 18 19]
[ 0  1  2  3  4  6  7  9 10 13 14 15 16 17 18 19] [ 5  8 11 12]


In [ ]:
for i,x in enumerate(train_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_train=X[ri:ri+size]
    Y_train=Y[ri:ri+size]
  else:
    X_train=np.concatenate((X_train,X[ri:ri+size]),axis=0)
    Y_train=np.concatenate((Y_train,Y[ri:ri+size]),axis=0)
for i,x in enumerate(val_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_test=X[ri:ri+size]
    Y_test=Y[ri:ri+size]
  else:
    X_test=np.concatenate((X_test,X[ri:ri+size]),axis=0)
    Y_test=np.concatenate((Y_test,Y[ri:ri+size]),axis=0)

##LSH Attention init

In [ ]:
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Keras-based multi-head attention layer."""


import collections
import math
import string

import numpy as np
import tensorflow as tf

from keras import constraints
from keras import initializers
from keras import regularizers
from keras.engine.base_layer import Layer
from keras.layers import activation
from keras.layers import core
from keras.layers import regularization
from keras.utils import tf_utils

# isort: off
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util.tf_export import keras_export

_CHR_IDX = string.ascii_lowercase

def _build_attention_equation(rank, attn_axes):
    """Builds einsum equations for the attention computation.
    Query, key, value inputs after projection are expected to have the shape as:
    `(bs, <non-attention dims>, <attention dims>, num_heads, channels)`.
    `bs` and `<non-attention dims>` are treated as `<batch dims>`.
    The attention operations can be generalized:
    (1) Query-key dot product:
    `(<batch dims>, <query attention dims>, num_heads, channels), (<batch dims>,
    <key attention dims>, num_heads, channels) -> (<batch dims>,
    num_heads, <query attention dims>, <key attention dims>)`
    (2) Combination:
    `(<batch dims>, num_heads, <query attention dims>, <key attention dims>),
    (<batch dims>, <value attention dims>, num_heads, channels) -> (<batch
    dims>, <query attention dims>, num_heads, channels)`
    Args:
      rank: Rank of query, key, value tensors.
      attn_axes: List/tuple of axes, `[-1, rank)`,
        that attention will be applied to.
    Returns:
      Einsum equations.
    """
    target_notation = _CHR_IDX[:rank]
    # `batch_dims` includes the head dim.
    batch_dims = tuple(np.delete(range(rank), attn_axes + (rank - 1,)))
    letter_offset = rank
    source_notation = ""
    for i in range(rank):
        if i in batch_dims or i == rank - 1:
            source_notation += target_notation[i]
        else:
            source_notation += _CHR_IDX[letter_offset]
            letter_offset += 1

    product_notation = "".join(
        [target_notation[i] for i in batch_dims]
        + [target_notation[i] for i in attn_axes]
        + [source_notation[i] for i in attn_axes]
    )
    dot_product_equation = "%s,%s->%s" % (
        source_notation,
        target_notation,
        product_notation,
    )
    attn_scores_rank = len(product_notation)
    combine_equation = "%s,%s->%s" % (
        product_notation,
        source_notation,
        target_notation,
    )
    return dot_product_equation, combine_equation, attn_scores_rank


def _build_proj_equation(free_dims, bound_dims, output_dims):
    """Builds an einsum equation for projections inside multi-head attention."""
    input_str = ""
    kernel_str = ""
    output_str = ""
    bias_axes = ""
    letter_offset = 0
    for i in range(free_dims):
        char = _CHR_IDX[i + letter_offset]
        input_str += char
        output_str += char

    letter_offset += free_dims
    for i in range(bound_dims):
        char = _CHR_IDX[i + letter_offset]
        input_str += char
        kernel_str += char

    letter_offset += bound_dims
    for i in range(output_dims):
        char = _CHR_IDX[i + letter_offset]
        kernel_str += char
        output_str += char
        bias_axes += char
    equation = f"{input_str},{kernel_str}->{output_str}"
    return equation, bias_axes, len(output_str)


def _get_output_shape(output_rank, known_last_dims):
    return [None] * (output_rank - len(known_last_dims)) + list(known_last_dims)

class Linformer(Layer):
    """MultiHeadAttention layer.
    This is an implementation of multi-headed attention as described in the
    paper "Attention is all you Need" (Vaswani et al., 2017).
    If `query`, `key,` `value` are the same, then
    this is self-attention. Each timestep in `query` attends to the
    corresponding sequence in `key`, and returns a fixed-width vector.
    This layer first projects `query`, `key` and `value`. These are
    (effectively) a list of tensors of length `num_attention_heads`, where the
    corresponding shapes are `(batch_size, <query dimensions>, key_dim)`,
    `(batch_size, <key/value dimensions>, key_dim)`,
    `(batch_size, <key/value dimensions>, value_dim)`.
    Then, the query and key tensors are dot-producted and scaled. These are
    softmaxed to obtain attention probabilities. The value tensors are then
    interpolated by these probabilities, then concatenated back to a single
    tensor.
    Finally, the result tensor with the last dimension as value_dim can take an
    linear projection and return.
    When using `MultiHeadAttention` inside a custom layer, the custom layer must
    implement its own `build()` method and call `MultiHeadAttention`'s
    `_build_from_signature()` there.
    This enables weights to be restored correctly when the model is loaded.
    Examples:
    Performs 1D cross-attention over two sequence inputs with an attention mask.
    Returns the additional attention weights over heads.
    >>> layer = MultiHeadAttention(num_heads=2, key_dim=2)
    >>> target = tf.keras.Input(shape=[8, 16])
    >>> source = tf.keras.Input(shape=[4, 16])
    >>> output_tensor, weights = layer(target, source,
    ...                                return_attention_scores=True)
    >>> print(output_tensor.shape)
    (None, 8, 16)
    >>> print(weights.shape)
    (None, 2, 8, 4)
    Performs 2D self-attention over a 5D input tensor on axes 2 and 3.
    >>> layer = MultiHeadAttention(
    ...     num_heads=2, key_dim=2, attention_axes=(2, 3))
    >>> input_tensor = tf.keras.Input(shape=[5, 3, 4, 16])
    >>> output_tensor = layer(input_tensor, input_tensor)
    >>> print(output_tensor.shape)
    (None, 5, 3, 4, 16)
    Args:
      num_heads: Number of attention heads.
      key_dim: Size of each attention head for query and key.
      value_dim: Size of each attention head for value.
      dropout: Dropout probability.
      use_bias: Boolean, whether the dense layers use bias vectors/matrices.
      output_shape: The expected shape of an output tensor, besides the batch
        and sequence dims. If not specified, projects back to the key feature
        dim.
      attention_axes: axes over which the attention is applied. `None` means
        attention over all axes, but batch, heads, and features.
      kernel_initializer: Initializer for dense layer kernels.
      bias_initializer: Initializer for dense layer biases.
      kernel_regularizer: Regularizer for dense layer kernels.
      bias_regularizer: Regularizer for dense layer biases.
      activity_regularizer: Regularizer for dense layer activity.
      kernel_constraint: Constraint for dense layer kernels.
      bias_constraint: Constraint for dense layer kernels.
    Call arguments:
      query: Query `Tensor` of shape `(B, T, dim)`.
      value: Value `Tensor` of shape `(B, S, dim)`.
      key: Optional key `Tensor` of shape `(B, S, dim)`. If not given, will use
        `value` for both `key` and `value`, which is the most common case.
      attention_mask: a boolean mask of shape `(B, T, S)`, that prevents
        attention to certain positions. The boolean mask specifies which query
        elements can attend to which key elements, 1 indicates attention and 0
        indicates no attention. Broadcasting can happen for the missing batch
        dimensions and the head dimension.
      return_attention_scores: A boolean to indicate whether the output should
        be `(attention_output, attention_scores)` if `True`, or
        `attention_output` if `False`. Defaults to `False`.
      training: Python boolean indicating whether the layer should behave in
        training mode (adding dropout) or in inference mode (no dropout).
        Defaults to either using the training mode of the parent layer/model,
        or False (inference) if there is no parent layer.
      use_causal_mask: A boolean to indicate whether to apply a causal mask to
        prevent tokens from attending to future tokens (e.g., used in a decoder
        Transformer).
    Returns:
      attention_output: The result of the computation, of shape `(B, T, E)`,
        where `T` is for target sequence shapes and `E` is the query input last
        dimension if `output_shape` is `None`. Otherwise, the multi-head outputs
        are projected to the shape specified by `output_shape`.
      attention_scores: [Optional] multi-head attention coefficients over
        attention axes.
    """

    def __init__(
        self,
        num_heads,
        key_dim,
        value_dim=None,
        dropout=0.0,
        use_bias=True,
        output_shape=None,
        attention_axes=None,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        k_value=32,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.supports_masking = True
        self._num_heads = num_heads
        self._key_dim = key_dim
        self._value_dim = value_dim if value_dim else key_dim
        self._dropout = dropout
        self._use_bias = use_bias
        self.k_value = k_value
        self._output_shape = output_shape
        self._kernel_initializer = initializers.get(kernel_initializer)
        self._bias_initializer = initializers.get(bias_initializer)
        self._kernel_regularizer = regularizers.get(kernel_regularizer)
        self._bias_regularizer = regularizers.get(bias_regularizer)
        self._activity_regularizer = regularizers.get(activity_regularizer)
        self._kernel_constraint = constraints.get(kernel_constraint)
        self._bias_constraint = constraints.get(bias_constraint)
        if attention_axes is not None and not isinstance(
            attention_axes, collections.abc.Sized
        ):
            self._attention_axes = (attention_axes,)
        else:
            self._attention_axes = attention_axes
        self._built_from_signature = False
        self._query_shape, self._key_shape, self._value_shape = None, None, None

    def get_config(self):
        config = {
            "num_heads": self._num_heads,
            "key_dim": self._key_dim,
            "value_dim": self._value_dim,
            "dropout": self._dropout,
            "use_bias": self._use_bias,
            "output_shape": self._output_shape,
            "attention_axes": self._attention_axes,
            "kernel_initializer": initializers.serialize(
                self._kernel_initializer
            ),
            "bias_initializer": initializers.serialize(self._bias_initializer),
            "kernel_regularizer": regularizers.serialize(
                self._kernel_regularizer
            ),
            "bias_regularizer": regularizers.serialize(self._bias_regularizer),
            "activity_regularizer": regularizers.serialize(
                self._activity_regularizer
            ),
            "kernel_constraint": constraints.serialize(self._kernel_constraint),
            "bias_constraint": constraints.serialize(self._bias_constraint),
            "query_shape": self._query_shape,
            "key_shape": self._key_shape,
            "value_shape": self._value_shape,
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        # If the layer has a different build() function from the Keras default,
        # we need to trigger the customized build to create weights.
        query_shape = config.pop("query_shape")
        key_shape = config.pop("key_shape")
        value_shape = config.pop("value_shape")
        layer = cls(**config)
        if None in [query_shape, key_shape, value_shape]:
            logging.warning(
                "One of dimensions of the input shape is missing. It "
                "should have been memorized when the layer was serialized. "
                "%s is created without weights.",
                str(cls),
            )
        else:
            layer._build_from_signature(query_shape, value_shape, key_shape)
        return layer

    def _build_from_signature(self, query, value, key=None):
        """Builds layers and variables.
        Once the method is called, self._built_from_signature will be set to
        True.
        Args:
          query: Query tensor or TensorShape.
          value: Value tensor or TensorShape.
          key: Key tensor or TensorShape.
        """
        self._built_from_signature = True
        if hasattr(query, "shape"):
            self._query_shape = tf.TensorShape(query.shape)
        else:
            self._query_shape = tf.TensorShape(query)
        if hasattr(value, "shape"):
            self._value_shape = tf.TensorShape(value.shape)
        else:
            self._value_shape = tf.TensorShape(value)
        if key is None:
            self._key_shape = self._value_shape
        elif hasattr(key, "shape"):
            self._key_shape = tf.TensorShape(key.shape)
        else:
            self._key_shape = tf.TensorShape(key)

        # Any setup work performed only once should happen in an `init_scope`
        # to avoid creating symbolic Tensors that will later pollute any eager
        # operations.
        with tf_utils.maybe_init_scope(self):
            free_dims = self._query_shape.rank - 1
            einsum_equation, bias_axes, output_rank = _build_proj_equation(
                free_dims, bound_dims=1, output_dims=2
            )
            self._query_dense = core.EinsumDense(
                einsum_equation,
                output_shape=_get_output_shape(
                    output_rank - 1, [self._num_heads, self._key_dim]
                ),
                bias_axes=bias_axes if self._use_bias else None,
                name="query",
                **self._get_common_kwargs_for_sublayer(),
            )
            einsum_equation, bias_axes, output_rank = _build_proj_equation(
                self._key_shape.rank - 1, bound_dims=1, output_dims=2
            )
            self._key_dense = core.EinsumDense(
                einsum_equation,
                output_shape=_get_output_shape(
                    output_rank - 1, [self._num_heads, self._key_dim]
                ),
                bias_axes=bias_axes if self._use_bias else None,
                name="key",
                **self._get_common_kwargs_for_sublayer(),
            )
            self._proj=tf.keras.layers.EinsumDense("bjhd,kj->bkhd",output_shape=(self.k_value,None,None))

            einsum_equation, bias_axes, output_rank = _build_proj_equation(
                self._value_shape.rank - 1, bound_dims=1, output_dims=2
            )
            self._value_dense = core.EinsumDense(
                einsum_equation,
                output_shape=_get_output_shape(
                    output_rank - 1, [self._num_heads, self._value_dim]
                ),
                bias_axes=bias_axes if self._use_bias else None,
                name="value",
                **self._get_common_kwargs_for_sublayer(),
            )

            # Builds the attention computations for multi-head dot product
            # attention.  These computations could be wrapped into the keras
            # attention layer once it supports mult-head einsum computations.
            self._build_attention(output_rank)
            self._output_dense = self._make_output_dense(
                free_dims,
                self._get_common_kwargs_for_sublayer(),
                "attention_output",
            )

    def _get_common_kwargs_for_sublayer(self):
        common_kwargs = dict(
            kernel_regularizer=self._kernel_regularizer,
            bias_regularizer=self._bias_regularizer,
            activity_regularizer=self._activity_regularizer,
            kernel_constraint=self._kernel_constraint,
            bias_constraint=self._bias_constraint,
        )
        # Create new clone of kernel/bias initializer, so that we don't reuse
        # the initializer instance, which could lead to same init value since
        # initializer is stateless.
        kernel_initializer = self._kernel_initializer.__class__.from_config(
            self._kernel_initializer.get_config()
        )
        bias_initializer = self._bias_initializer.__class__.from_config(
            self._bias_initializer.get_config()
        )
        common_kwargs["kernel_initializer"] = kernel_initializer
        common_kwargs["bias_initializer"] = bias_initializer
        return common_kwargs

    def _make_output_dense(self, free_dims, common_kwargs, name=None):
        """Builds the output projection matrix.
        Args:
          free_dims: Number of free dimensions for einsum equation building.
          common_kwargs: Common keyword arguments for einsum layer.
          name: Name for the projection layer.
        Returns:
          Projection layer.
        """
        if self._output_shape:
            if not isinstance(self._output_shape, collections.abc.Sized):
                output_shape = [self._output_shape]
            else:
                output_shape = self._output_shape
        else:
            output_shape = [self._query_shape[-1]]
        einsum_equation, bias_axes, output_rank = _build_proj_equation(
            free_dims, bound_dims=2, output_dims=len(output_shape)
        )
        return core.EinsumDense(
            einsum_equation,
            output_shape=_get_output_shape(output_rank - 1, output_shape),
            bias_axes=bias_axes if self._use_bias else None,
            name=name,
            **common_kwargs,
        )

    def _build_attention(self, rank):
        """Builds multi-head dot-product attention computations.
        This function builds attributes necessary for `_compute_attention` to
        customize attention computation to replace the default dot-product
        attention.
        Args:
          rank: the rank of query, key, value tensors.
        """
        if self._attention_axes is None:
            self._attention_axes = tuple(range(1, rank - 2))
        else:
            self._attention_axes = tuple(self._attention_axes)
        (
            self._dot_product_equation,
            self._combine_equation,
            attn_scores_rank,
        ) = _build_attention_equation(rank, attn_axes=self._attention_axes)
        norm_axes = tuple(
            range(
                attn_scores_rank - len(self._attention_axes), attn_scores_rank
            )
        )
        self._softmax = activation.Softmax(axis=norm_axes)
        self._dropout_layer = regularization.Dropout(rate=self._dropout)

    def _masked_softmax(self, attention_scores, attention_mask=None):
        # Normalize the attention scores to probabilities.
        # `attention_scores` = [B, N, T, S]
        if attention_mask is not None:
            # The expand dim happens starting from the `num_heads` dimension,
            # (<batch_dims>, num_heads, <query_attention_dims,
            # key_attention_dims>)
            mask_expansion_axis = -len(self._attention_axes) * 2 - 1
            for _ in range(
                len(attention_scores.shape) - len(attention_mask.shape)
            ):
                attention_mask = tf.expand_dims(
                    attention_mask, axis=mask_expansion_axis
                )
        return self._softmax(attention_scores, attention_mask)

    def _compute_attention(
        self, query, key, value, attention_mask=None, training=None
    ):
        """Applies Dot-product attention with query, key, value tensors.
        This function defines the computation inside `call` with projected
        multi-head Q, K, V inputs. Users can override this function for
        customized attention implementation.
        Args:
          query: Projected query `Tensor` of shape `(B, T, N, key_dim)`.
          key: Projected key `Tensor` of shape `(B, S, N, key_dim)`.
          value: Projected value `Tensor` of shape `(B, S, N, value_dim)`.
          attention_mask: a boolean mask of shape `(B, T, S)`, that prevents
            attention to certain positions. It is generally not needed if the
            `query` and `value` (and/or `key`) are masked.
          training: Python boolean indicating whether the layer should behave in
            training mode (adding dropout) or in inference mode (doing nothing).
        Returns:
          attention_output: Multi-headed outputs of attention computation.
          attention_scores: Multi-headed attention weights.
        """
        # Note: Applying scalar multiply at the smaller end of einsum improves
        # XLA performance, but may introduce slight numeric differences in
        # the Transformer attention head.
        query = tf.multiply(query, 1.0 / math.sqrt(float(self._key_dim)))
        # Take the dot product between "query" and "key" to get the raw
        # attention scores.
        attention_scores = tf.einsum(self._dot_product_equation, key, query)
        attention_scores = self._masked_softmax(
            attention_scores, attention_mask
        )
        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_scores_dropout = self._dropout_layer(
            attention_scores, training=training
        )

        # `context_layer` = [B, T, N, H]
        attention_output = tf.einsum(
            self._combine_equation, attention_scores_dropout, value
        )
        #import sys
        #sys.exit()
        return attention_output, attention_scores

    def call(
        self,
        query,
        value,
        key=None,
        attention_mask=None,
        return_attention_scores=False,
        training=None,
        use_causal_mask=False,
    ):
        attention_mask = self._compute_attention_mask(
            query,
            value,
            key=key,
            attention_mask=attention_mask,
            use_causal_mask=use_causal_mask,
        )

        if not self._built_from_signature:
            self._build_from_signature(query=query, value=value, key=key)
        if key is None:
            key = value

        query_is_ragged = isinstance(query, tf.RaggedTensor)
        if query_is_ragged:
            query_lengths = query.nested_row_lengths()
            query = query.to_tensor()

        key_is_ragged = isinstance(key, tf.RaggedTensor)
        value_is_ragged = isinstance(value, tf.RaggedTensor)
        if key_is_ragged and value_is_ragged:
            # Ensure they have the same shape.
            bounding_shape = tf.math.maximum(
                key.bounding_shape(), value.bounding_shape()
            )
            key = key.to_tensor(shape=bounding_shape)
            value = value.to_tensor(shape=bounding_shape)
        elif key_is_ragged:
            key = key.to_tensor(shape=tf.shape(value))
        elif value_is_ragged:
            value = value.to_tensor(shape=tf.shape(key))

        #   N = `num_attention_heads`
        #   H = `size_per_head`
        # `query` = [B, T, N ,H]
        query = self._query_dense(query)
        # `key` = [B, S, N, H]
        key = self._key_dense(key)
        key = self._proj(key)
        # `value` = [B, S, N, H]
        value = self._value_dense(value)
        value = self._proj(value)
        attention_output, attention_scores = self._compute_attention(
            query, key, value, attention_mask, training
        )
        attention_output = self._output_dense(attention_output)

        if query_is_ragged:
            attention_output = tf.RaggedTensor.from_tensor(
                attention_output, lengths=query_lengths
            )

        if return_attention_scores:
            return attention_output, attention_scores
        return attention_output

    def _compute_attention_mask(
        self, query, value, key=None, attention_mask=None, use_causal_mask=False
    ):
        """Computes the attention mask, using the Keras masks of the inputs.
        * The `query`'s mask is reshaped from [B, T] to [B, T, 1].
        * The `value`'s mask is reshaped from [B, S] to [B, 1, S].
        * The `key`'s mask is reshaped from [B, S] to [B, 1, S]. The `key`'s
          mask is ignored if `key` is `None` or if `key is value`.
        * If `use_causal_mask=True`, then the causal mask is computed. Its shape
          is [1, T, S].
        All defined masks are merged using a logical AND operation (`&`).
        In general, if the `query` and `value` are masked, then there is no need
        to define the `attention_mask`.
        Args:
          query: Projected query `Tensor` of shape `(B, T, N, key_dim)`.
          key: Projected key `Tensor` of shape `(B, T, N, key_dim)`.
          value: Projected value `Tensor` of shape `(B, T, N, value_dim)`.
          attention_mask: a boolean mask of shape `(B, T, S)`, that prevents
            attention to certain positions.
          use_causal_mask: A boolean to indicate whether to apply a causal mask
            to prevent tokens from attending to future tokens (e.g., used in a
            decoder Transformer).
        Returns:
          attention_mask: a boolean mask of shape `(B, T, S)`, that prevents
            attention to certain positions, based on the Keras masks of the
            `query`, `key`, `value`, and `attention_mask` tensors, and the
            causal mask if `use_causal_mask=True`.
        """
        query_mask = getattr(query, "_keras_mask", None)
        value_mask = getattr(value, "_keras_mask", None)
        key_mask = getattr(key, "_keras_mask", None)
        auto_mask = None
        if query_mask is not None:
            query_mask = tf.cast(query_mask, tf.bool)  # defensive casting
            # B = batch size, T = max query length
            auto_mask = query_mask[:, :, tf.newaxis]  # shape is [B, T, 1]
        if value_mask is not None:
            value_mask = tf.cast(value_mask, tf.bool)  # defensive casting
            # B = batch size, S == max value length
            mask = value_mask[:, tf.newaxis, :]  # shape is [B, 1, S]
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if key_mask is not None:
            key_mask = tf.cast(key_mask, tf.bool)  # defensive casting
            # B == batch size, S == max key length == max value length
            mask = key_mask[:, tf.newaxis, :]  # shape is [B, 1, S]
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if use_causal_mask:
            # the shape of the causal mask is [1, T, S]
            mask = self._compute_causal_mask(query, value)
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if auto_mask is not None:
            # merge attention_mask & automatic mask, to shape [B, T, S]
            attention_mask = (
                auto_mask
                if attention_mask is None
                else tf.cast(attention_mask, bool) & auto_mask
            )
        return attention_mask

    def _compute_causal_mask(self, query, value=None):
        """Computes a causal mask (e.g., for masked self-attention layers).
        For example, if query and value both contain sequences of length 4,
        this function returns a boolean `Tensor` equal to:
        ```
        [[[True,  False, False, False],
          [True,  True,  False, False],
          [True,  True,  True,  False],
          [True,  True,  True,  True]]]
        ```
        Args:
          query: query `Tensor` of shape `(B, T, ...)`.
          value: value `Tensor` of shape `(B, S, ...)` (optional, defaults to
          query).
        Returns:
          mask: a boolean `Tensor` of shape [1, T, S] containing a lower
                triangular matrix of shape [T, S].
        """
        q_seq_length = tf.shape(query)[1]
        v_seq_length = q_seq_length if value is None else tf.shape(value)[1]
        return tf.linalg.band_part(  # creates a lower triangular matrix
            tf.ones((1, q_seq_length, v_seq_length), tf.bool), -1, 0
        )

    def compute_output_shape(self, query_shape, value_shape, key_shape=None):

        if key_shape is None:
            key_shape = value_shape

        query_shape = tf.TensorShape(query_shape)
        value_shape = tf.TensorShape(value_shape)
        key_shape = tf.TensorShape(key_shape)

        if query_shape[-1] != value_shape[-1]:
            raise ValueError(
                "The last dimension of `query_shape` and `value_shape` "
                f"must be equal, but are {query_shape[-1]}, {value_shape[-1]}. "
                "Received: query_shape={query_shape}, value_shape={value_shape}"
            )

        if value_shape[1:-1] != key_shape[1:-1]:
            raise ValueError(
                "All dimensions of `value` and `key`, except the last one, "
                f"must be equal. Received {value_shape} and "
                f"{key_shape}"
            )

        if self._output_shape:
            return query_shape[:-1].concatenate(self._output_shape)

        return query_shape

##LSH Attention as keras multihead subclass

In [ ]:
import collections
import math
import string
import numpy as np
import tensorflow as tf
from keras.layers.attention.multi_head_attention import MultiHeadAttention
from keras.utils import tf_utils




class LSHAttention(MultiHeadAttention):
  def __init__(self, *args, **kwargs):
    self.n_buckets= kwargs.pop('n_buckets', 4)
    super().__init__(*args, **kwargs)
  def _build_from_signature(self, query, value, key=None):
    with tf_utils.maybe_init_scope(self):
      self._hash=tf.keras.layers.EinsumDense("bjhd,dc->bjhc",output_shape=(None,None,self.n_buckets // 2))
      self.lsh_proj=tf.random.normal
    super()._build_from_signature(query, value, key=None)

  def call(
        self,
        query,
        value,
        key=None,
        attention_mask=None,
        return_attention_scores=False,
        training=None,
        use_causal_mask=False,
        ):
        attention_mask = self._compute_attention_mask(
            query,
            value,
            key=key,
            attention_mask=attention_mask,
            use_causal_mask=use_causal_mask,
        )

        if not self._built_from_signature:
            self._build_from_signature(query=query, value=value, key=key)
        if key is None:
            key = value

        query_is_ragged = isinstance(query, tf.RaggedTensor)
        if query_is_ragged:
            query_lengths = query.nested_row_lengths()
            query = query.to_tensor()

        key_is_ragged = isinstance(key, tf.RaggedTensor)
        value_is_ragged = isinstance(value, tf.RaggedTensor)
        if key_is_ragged and value_is_ragged:
            # Ensure they have the same shape.
            bounding_shape = tf.math.maximum(
                key.bounding_shape(), value.bounding_shape()
            )
            key = key.to_tensor(shape=bounding_shape)
            value = value.to_tensor(shape=bounding_shape)
        elif key_is_ragged:
            key = key.to_tensor(shape=tf.shape(value))
        elif value_is_ragged:
            value = value.to_tensor(shape=tf.shape(key))

        #   N = `num_attention_heads`
        #   H = `size_per_head`
        # `query` = [B, T, N ,H]
        query = self._query_dense(query)
        # `key` = [B, S, N, H]
        key = query
        # `value` = [B, S, N, H]
        value = self._value_dense(value)
        for i in range(value.shape[2]):
          #here we make the rotation matrix, that will hash the vectors
          random_rot=self.lsh_proj([value.shape[-1],self.n_buckets // 2])
          #hash=self._hash(value)

          #here we make the rotation using dot product, and making the buckets
          hash=tf.matmul(value[:,:,i,:],random_rot)
          hash=tf.concat([hash,-hash],axis=-1)

          #here we make each token be clasified to a bucket
          buckets=tf.math.argmax(hash,axis=-1)

          #This is the index that will sort the inputs, and is made by sorting the buckets
          index=tf.argsort(buckets,axis=-1)

          #here we sort the main inputs usinf the index
          sv=tf.gather(value[:,:,i,:],index,axis=1,batch_dims=-1)
          sq=sk=tf.gather(query[:,:,i,:],index,axis=1,batch_dims=-1)
          #And chunk the inputs based on how many buckets we have
          sv=tf.split(sv, self.n_buckets, axis=1)
          sq=tf.split(sq, self.n_buckets, axis=1)
          j=0
          for v,q in zip(sv,sq):
            attention_output, attention_scores = self._compute_attention(
                q[:,:,None,:], q[:,:,None,:], v[:,:,None,:], attention_mask, training
            )
            if j==0:
              att=attention_output
            else:
              att=tf.concat([att,attention_output],axis=1)
            j+=1
          attention_output=att
          if i==0:
            att_o=attention_output
          else:
            att_o=tf.concat([att_o,attention_output],axis=2)
        attention_output = att_o
        index=tf.argsort(index,axis=-1)
        attention_output = tf.gather(attention_output, index,axis=1,batch_dims=-1)
        attention_output = self._output_dense(attention_output)
        if query_is_ragged:
            attention_output = tf.RaggedTensor.from_tensor(
                attention_output, lengths=query_lengths
        )
        if return_attention_scores:
            return attention_output, attention_scores
        return attention_output

In [ ]:
import collections
import math
import string
import numpy as np
import tensorflow as tf
from keras.layers.attention.multi_head_attention import MultiHeadAttention
from keras.utils import tf_utils




class LSHAttention(MultiHeadAttention):
  def __init__(self, *args, **kwargs):
    self.n_buckets= kwargs.pop('n_buckets', 4)
    super().__init__(*args, **kwargs)
  def _build_from_signature(self, query, value, key=None):
    with tf_utils.maybe_init_scope(self):
      self.lsh_proj=tf.random.normal
    super()._build_from_signature(query, value, key=None)

  def call(
        self,
        query,
        value,
        key=None,
        attention_mask=None,
        return_attention_scores=False,
        training=None,
        use_causal_mask=False,
        ):
        attention_mask = self._compute_attention_mask(
            query,
            value,
            key=key,
            attention_mask=attention_mask,
            use_causal_mask=use_causal_mask,
        )

        if not self._built_from_signature:
            self._build_from_signature(query=query, value=value, key=key)
        if key is None:
            key = value

        query_is_ragged = isinstance(query, tf.RaggedTensor)
        if query_is_ragged:
            query_lengths = query.nested_row_lengths()
            query = query.to_tensor()

        key_is_ragged = isinstance(key, tf.RaggedTensor)
        value_is_ragged = isinstance(value, tf.RaggedTensor)
        if key_is_ragged and value_is_ragged:
            # Ensure they have the same shape.
            bounding_shape = tf.math.maximum(
                key.bounding_shape(), value.bounding_shape()
            )
            key = key.to_tensor(shape=bounding_shape)
            value = value.to_tensor(shape=bounding_shape)
        elif key_is_ragged:
            key = key.to_tensor(shape=tf.shape(value))
        elif value_is_ragged:
            value = value.to_tensor(shape=tf.shape(key))

        #   N = `num_attention_heads`
        #   H = `size_per_head`
        # `query` = [B, T, N ,H]
        query = self._query_dense(query)
        # `key` = [B, S, N, H]
        key = query
        # `value` = [B, S, N, H]
        value = self._value_dense(value)
        for i in range(value.shape[2]):
          #here we make the rotation matrix, that will hash the vectors
          random_rot=self.lsh_proj([value.shape[-1],self.n_buckets // 2])
          #hash=self._hash(value)

          #here we make the rotation using dot product, and making the buckets
          hash=tf.matmul(value[:,:,i,:],random_rot)
          hash=tf.concat([hash,-hash],axis=-1)

          #here we make each token be clasified to a bucket
          buckets=tf.math.argmax(hash,axis=-1)

          #This is the index that will sort the inputs, and is made by sorting the buckets
          index=tf.argsort(buckets,axis=-1)

          #here we sort the main inputs usinf the index
          sv=tf.gather(value[:,:,i,:],index,axis=1,batch_dims=-1)
          sq=sk=tf.gather(query[:,:,i,:],index,axis=1,batch_dims=-1)
          #And chunk the inputs based on how many buckets we have

          j=0
          chunk_size=int(sv.shape[1]/self.n_buckets)
          for j in range(self.n_buckets):
            q=sq[:,j*chunk_size:(j+1)*chunk_size,:]
            v=sv[:,j*chunk_size:(j+1)*chunk_size,:]
            attention_output, attention_scores = self._compute_attention(
                q[:,:,None,:], q[:,:,None,:], v[:,:,None,:], attention_mask, training
            )
            if j==0:
              att=attention_output
            else:
              att=tf.concat([att,attention_output],axis=1)
          attention_output=att
          if i==0:
            att_o=attention_output
          else:
            att_o=tf.concat([att_o,attention_output],axis=2)
        attention_output = att_o
        attention_output = self._output_dense(attention_output)
        if query_is_ragged:
            attention_output = tf.RaggedTensor.from_tensor(
                attention_output, lengths=query_lengths
        )
        if return_attention_scores:
            return attention_output, attention_scores
        return attention_output

##Patch wise

In [ ]:
from keras.layers import Layer
import keras.backend as K
import torch.nn as nn
import tensorflow as tf

In [ ]:
def DiceMetric(y_true, y_pred):
  smooth=1e-6
  gama=2
  y_true, y_pred = tf.cast(
      y_true, dtype=tf.float32), tf.cast(y_pred, tf.float32)
  nominator = 2 * \
      tf.reduce_sum(tf.multiply(y_pred, y_true)) + smooth
  denominator = tf.reduce_sum(
      y_pred ** gama) + tf.reduce_sum(y_true ** gama) + smooth
  result = tf.divide(nominator, denominator)
  return result
def DiceLoss(y_true, y_pred):
      result= 1- DiceMetric(y_true, y_pred)
      return result

In [ ]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou
def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

In [ ]:
def dice_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      #None,128,128,1
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      #None,128,128
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += dice_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def iou_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += iou_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def conv_block(X,f,d=0.1,group=1):
  c = tf.keras.layers.Conv2D(f[0], (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',groups=group)(X)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[1], (3, 3), kernel_initializer='he_normal', padding='same', groups=group)(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  s = tf.keras.layers.Conv2D(f[1], (1, 1), kernel_initializer='he_normal', padding='same')(X)
  s = tf.keras.layers.BatchNormalization(axis=3)(s)
  c = tf.keras.layers.Add()([c,s])
  c = tf.keras.layers.ReLU()(c)
  return c,c

In [ ]:
import sys

In [ ]:
def Flatten(M):
  s1=M.shape[1]
  s2=M.shape[2]
  s3=M.shape[3]
  flat=s1*s2
  M= tf.reshape(M,[-1,flat,s3])
  return M,s1,s2,s3

In [ ]:
def Flatten_LSH(M,N,nheads,dimk,k=128):
  M,s1,s2,s3= Flatten(M)
  N,_,_,_= Flatten(N)
  pm=LSHAttention(num_heads=nheads,key_dim=dimk,dropout=0.2,n_buckets=k)(M,M)
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
def vit(X,Y,n_heads=16,psize=16,k=256,dim="4D"):
  npatch=int(X.shape[1]/psize)
  psizey=int(psize/2)
  kdim=int(X.shape[3]/n_heads)
  Y=tf.keras.layers.Dense(X.shape[3])(Y)
  if npatch==0:
    pm=Flatten_LSH(X,Y,n_heads,kdim,k)
    return pm
  for i in range(npatch):
    st=i*psize
    sty=i*psizey
    end=st+psize
    endy=sty+psizey
    for j in range(npatch):
      st2=j*psize
      end2=st2+psize
      sty2=j*psizey
      endy2=sty2+psizey
      patch= X[:,st:end,st2:end2,:]
      patch2= Y[:,sty:endy,sty2:endy2,:]
      pm=Flatten_LSH(patch,patch2,n_heads,kdim,k)
      if j==0:
        A=pm
      else:
        A=tf.concat([A,pm],2)
    if i==0:
      V=A
    else:
      V=tf.concat([V,A],1)
  return V



### Vit

In [ ]:
import tensorflow as tf

def create_model():
  nheads=4
  sizep=32
  gr=4
  kval=4
  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path
  m1= vit(z4,z5,nheads,sizep,k=kval)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= vit(z3,z6,nheads,sizep,k=kval)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= vit(z2,z7,nheads,sizep,k=kval)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= vit(z1,z8,nheads,sizep,k=kval)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###LSH without Patches

In [ ]:
def Flatten(M):
  s1=M.shape[1]
  s2=M.shape[2]
  s3=M.shape[3]
  flat=s1*s2
  M= tf.reshape(M,[-1,flat,s3])
  return M,s1,s2,s3

In [ ]:
def LSH_lay(M,nheads,k=128,nbuckets=4):
  M,s1,s2,s3= Flatten(M)
  dimk=int(M.shape[2]/nheads)
  pm=LSHAttention(num_heads=nheads,key_dim=dimk,dropout=0.2,n_buckets=nbuckets)(M,M)
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
import tensorflow as tf

def create_model():
  nheads=4
  sizep=32
  gr=4

  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path

  m1= LSH_lay(z4,nheads)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= LSH_lay(z3,nheads,nbuckets=8)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= LSH_lay(z2,nheads,nbuckets=8)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= LSH_lay(z1,nheads,nbuckets=32)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###Test

In [ ]:
model=create_model()
model.fit(X_train,Y_train,batch_size=16,epochs=25)

Epoch 1/25
158/158 [==============================] - 454s 875ms/step - loss: 0.2368 - accuracy: 0.9364 - iou_coef_multilabel: 0.3757 - dice_coef_multilabel: 0.4452
Epoch 2/25
158/158 [==============================] - 127s 806ms/step - loss: 0.0523 - accuracy: 0.9863 - iou_coef_multilabel: 0.5403 - dice_coef_multilabel: 0.5984
Epoch 3/25
158/158 [==============================] - 127s 803ms/step - loss: 0.0356 - accuracy: 0.9898 - iou_coef_multilabel: 0.5898 - dice_coef_multilabel: 0.6427
Epoch 4/25
158/158 [==============================] - 127s 807ms/step - loss: 0.0286 - accuracy: 0.9913 - iou_coef_multilabel: 0.6189 - dice_coef_multilabel: 0.6692
Epoch 5/25
158/158 [==============================] - 127s 802ms/step - loss: 0.0245 - accuracy: 0.9922 - iou_coef_multilabel: 0.6402 - dice_coef_multilabel: 0.6887
Epoch 6/25
158/158 [==============================] - 127s 803ms/step - loss: 0.0221 - accuracy: 0.9928 - iou_coef_multilabel: 0.6557 - dice_coef_multilabel: 0.7027
Epoch 7/25

for split version, time for execution 42 m

patched version time for execution 34 m

In [ ]:
scores= model.evaluate(X_test, Y_test, verbose=0)
print(f'Score for fold {0}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')

Score for fold 0: loss of 0.023588653653860092; accuracy of 99.25824999809265% iou_coef_multilabel of 75.87804794311523% dice_coef_multilabel of 79.55738306045532%


Patch size=32 k=32:

Score for fold 0: loss of 0.02660856954753399; accuracy of 99.21053647994995% iou_coef_multilabel of 73.97453188896179% dice_coef_multilabel of 77.59955525398254%

No Patch k=128:

Score for fold 0: loss of 0.03675605729222298; accuracy of 98.8860547542572% iou_coef_multilabel of 69.78949904441833% dice_coef_multilabel of 74.2724359035492%

Patch size=32 k=256:
Score for fold 0: loss of 0.11757280677556992; accuracy of 96.79576754570007% iou_coef_multilabel of 71.6090202331543% dice_coef_multilabel of 79.81376647949219%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

####Patch test

k=32

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.029409149661660194; accuracy of 98.91234040260315% iou_coef_multilabel of 70.78478932380676% dice_coef_multilabel of 75.16862750053406%
Score for fold 2: loss of 0.042917650192976; accuracy of 98.51773381233215% iou_coef_multilabel of 70.23463249206543% dice_coef_multilabel of 75.7128119468689%
Score for fold 3: loss of 0.04444853588938713; accuracy of 98.51000308990479% iou_coef_multilabel of 67.47883558273315% dice_coef_multilabel of 72.66407012939453%
Score for fold 4: loss of 0.10042478889226913; accuracy of 96.02788090705872% iou_coef_multilabel of 55.91818690299988% dice_coef_multilabel of 63.355553150177%
Score for fold 5: loss of 0.03272783383727074; accuracy of 98.82147312164307% iou_coef_multilabel of 69.75324749946594% dice_coef_multilabel of 74.68957304954529%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.025224272161722183; accuracy of 99.0780770778656% iou_coef_multilabel of 75.7142961025238% dice_coef_multilabel of 79.58106994628906%
Score for fold 2: loss of 0.04713413491845131; accuracy of 98.5433280467987% iou_coef_multilabel of 74.99583959579468% dice_coef_multilabel of 80.19270896911621%
Score for fold 3: loss of 0.04239453375339508; accuracy of 98.56003522872925% iou_coef_multilabel of 71.22631072998047% dice_coef_multilabel of 76.35141015052795%
Score for fold 4: loss of 0.04546702280640602; accuracy of 98.31485748291016% iou_coef_multilabel of 70.64926028251648% dice_coef_multilabel of 77.11712121963501%
Score for fold 5: loss of 0.032865818589925766; accuracy of 98.83249998092651% iou_coef_multilabel of 71.10702991485596% dice_coef_multilabel of 75.85053443908691%


k=256

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.016601325944066048; accuracy of 99.49246644973755% iou_coef_multilabel of 80.1440417766571% dice_coef_multilabel of 82.72783160209656%
Score for fold 2: loss of 0.02792888507246971; accuracy of 99.11463856697083% iou_coef_multilabel of 77.49691009521484% dice_coef_multilabel of 81.86010122299194%
Score for fold 3: loss of 0.033090461045503616; accuracy of 99.0357756614685% iou_coef_multilabel of 75.54389238357544% dice_coef_multilabel of 79.85571622848511%
Score for fold 4: loss of 0.02735254541039467; accuracy of 99.02375340461731% iou_coef_multilabel of 75.38833618164062% dice_coef_multilabel of 80.59991002082825%
Score for fold 5: loss of 0.02401789091527462; accuracy of 99.26612973213196% iou_coef_multilabel of 76.66497826576233% dice_coef_multilabel of 80.13312816619873%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01717947982251644; accuracy of 99.4581937789917% iou_coef_multilabel of 79.65168952941895% dice_coef_multilabel of 82.61964917182922%
Score for fold 2: loss of 0.03299105167388916; accuracy of 99.06346797943115% iou_coef_multilabel of 78.3797025680542% dice_coef_multilabel of 82.73652791976929%
Score for fold 3: loss of 0.034237224608659744; accuracy of 99.15516972541809% iou_coef_multilabel of 78.44327688217163% dice_coef_multilabel of 82.67260193824768%


KeyboardInterrupt: ignored

####No Patch test

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.02731938473880291; accuracy of 99.00254607200623% iou_coef_multilabel of 71.55993580818176% dice_coef_multilabel of 75.51640272140503%
Score for fold 2: loss of 0.04416176304221153; accuracy of 98.46460819244385% iou_coef_multilabel of 70.76845169067383% dice_coef_multilabel of 76.32638216018677%
Score for fold 3: loss of 0.04237723350524902; accuracy of 98.45744967460632% iou_coef_multilabel of 67.4326241016388% dice_coef_multilabel of 72.71857857704163%
Score for fold 4: loss of 0.04793655872344971; accuracy of 98.23458194732666% iou_coef_multilabel of 68.04465651512146% dice_coef_multilabel of 74.47894811630249%
Score for fold 5: loss of 0.0348425917327404; accuracy of 98.7302839756012% iou_coef_multilabel of 65.84467887878418% dice_coef_multilabel of 70.64203023910522%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

KeyboardInterrupt: ignored

##Model data augmented kfold

In [ ]:
import tensorflow as tf
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("vertical"),
  tf.keras.layers.RandomRotation(0.1),
])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  train_data = tf.data.Dataset.from_tensor_slices((X_fold, Y_fold))
  # Apply the data augmentation pipeline to both X_train and Y_train
  augmented_train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x, y: x)
  Y_train_augmented = augmented_train_data.map(lambda x, y: y)

  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = np.asarray(list(Y_train_augmented.as_numpy_iterator()))

  X_train_combined = np.concatenate([X_fold, X_train_augmented], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  train_data = tf.data.Dataset.from_tensor_slices((X_fold, Y_fold))
  # Apply the data augmentation pipeline to both X_train and Y_train
  augmented_train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x, y: x)
  Y_train_augmented = augmented_train_data.map(lambda x, y: y)

  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = np.asarray(list(Y_train_augmented.as_numpy_iterator()))

  X_train_combined = np.concatenate([X_fold, X_train_augmented], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.06248067319393158; accuracy of 97.74013757705688% iou_coef_multilabel of 69.58065629005432% dice_coef_multilabel of 75.24264454841614%
Score for fold 2: loss of 0.07446722686290741; accuracy of 97.424978017807% iou_coef_multilabel of 66.99366569519043% dice_coef_multilabel of 73.48106503486633%
Score for fold 3: loss of 0.07256180793046951; accuracy of 97.48387932777405% iou_coef_multilabel of 64.70680236816406% dice_coef_multilabel of 70.94796895980835%
Score for fold 4: loss of 0.06639942526817322; accuracy of 97.35667705535889% iou_coef_multilabel of 60.659801959991455% dice_coef_multilabel of 67.65385866165161%
Score for fold 5: loss of 0.06580083817243576; accuracy of 97.68750071525574% iou_coef_multilabel of 59.561049938201904% dice_coef_multilabel of 65.57979583740234%


##Model data augmented kfold Fixed

In [ ]:
import tensorflow as tf
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("vertical"),
  tf.keras.layers.RandomRotation(0.1),
])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  X__2 = np.concatenate([X_fold, Y_fold], axis=-1)
  X__2 = tf.data.Dataset.from_tensor_slices((X__2))
  augmented_train_data = X__2.map(lambda x: data_augmentation(x, training=True))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x: x)
  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = X_train_augmented[:,:,:,1:5]
  X_train_augmented = X_train_augmented[:,:,:,0]

  X_train_combined = np.concatenate([X_fold, X_train_augmented[:,:,:,None]], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01390768215060234; accuracy of 99.5705783367157% iou_coef_multilabel of 86.49618029594421% dice_coef_multilabel of 89.0194833278656%
Score for fold 2: loss of 0.03675416111946106; accuracy of 98.97904992103577% iou_coef_multilabel of 79.32537794113159% dice_coef_multilabel of 83.57661962509155%
Score for fold 3: loss of 0.029099980369210243; accuracy of 99.24059510231018% iou_coef_multilabel of 80.83962798118591% dice_coef_multilabel of 84.87486839294434%
Score for fold 4: loss of 0.023593006655573845; accuracy of 99.0794837474823% iou_coef_multilabel of 81.77911639213562% dice_coef_multilabel of 86.97380423545837%


KeyboardInterrupt: ignored

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  X__2 = np.concatenate([X_fold, Y_fold], axis=-1)
  X__2 = tf.data.Dataset.from_tensor_slices((X__2))
  augmented_train_data = X__2.map(lambda x: data_augmentation(x, training=True))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x: x)
  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = X_train_augmented[:,:,:,1:5]
  X_train_augmented = X_train_augmented[:,:,:,0]

  X_train_combined = np.concatenate([X_fold, X_train_augmented[:,:,:,None]], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.013969536870718002; accuracy of 99.5184600353241% iou_coef_multilabel of 78.77731919288635% dice_coef_multilabel of 81.8413496017456%
Score for fold 2: loss of 0.029770037159323692; accuracy of 99.06414747238159% iou_coef_multilabel of 80.31871914863586% dice_coef_multilabel of 84.64120626449585%
Score for fold 3: loss of 0.026543816551566124; accuracy of 99.26268458366394% iou_coef_multilabel of 81.27678036689758% dice_coef_multilabel of 85.34112572669983%
Score for fold 4: loss of 0.02282973751425743; accuracy of 99.13150072097778% iou_coef_multilabel of 81.2670886516571% dice_coef_multilabel of 86.49913668632507%
Score for fold 5: loss of 0.02242366410791874; accuracy of 99.32538270950317% iou_coef_multilabel of 75.33347010612488% dice_coef_multilabel of 78.96603345870972%


#LSH attention with attention between chunks as keras multihead subclass

In [ ]:
!gdown "1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t
To: /content/input.zip
100% 367M/367M [00:02<00:00, 159MB/s]


In [ ]:
!unzip -q input.zip

replace input/train/masks/coronacases_009_38.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 1

In [ ]:
import nibabel as nib
import os
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import cv2
import torch

In [ ]:
import imageio as iio
import glob
from skimage.transform import resize
import PIL
from PIL import Image
src="/content/input/train"
imag="/images/"
X=np.zeros((len(glob.glob(src+imag+"*.png")),IMG_WIDTH,IMG_HEIGHT,1))
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x!="/content/input/train/images/coronacases_009_149.png":
    im=Image.open(x)
    X[i,:,:,0]=im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST)
mas="/Amasks/"
Y=np.zeros((len(glob.glob(src+mas+"*.png")),IMG_WIDTH,IMG_HEIGHT))
for i,x in enumerate(sorted(glob.glob(src+mas+"*.png"))):
  if x!="/content/input/train/Amasks/coronacases_009_149.png":
    im=Image.open(x)
    im=np.array(im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST))
    im[im==85]=1
    im[im==170]=2
    im[im==255]=3
    Y[i,:,:]=im
import tensorflow as tf
print(Y.shape)
print(np.unique(Y))
Y=tf.keras.utils.to_categorical(Y)
print(Y.shape)
print(np.unique(Y))

(3520, 128, 128)
[0. 1. 2. 3.]
(3520, 128, 128, 4)
[0. 1.]


In [ ]:
src="/content/input/train"
imag="/images/"
Siz=0
id=0
ri=0
metadat=[]
c="001"
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x.find("radiopaedia") == -1:
    x=x[x.find("_")+1:]
    x=x[:x.find("_")]
  else:
    x=x[x.find("_")+1:]
    x=x[:x.find("_",9)]
  if c!=x:
    print(x)
    metadat.append([id,Siz,ri])
    ri=i
    c=x
    id+=1
    Siz=0
  Siz+=1
metadat.append([id,Siz,ri])

002
003
004
005
006
007
008
009
010
10_85902_1
10_85902_3
14_85914_0
27_86410_0
29_86490_1
29_86491_1
36_86526_0
40_86625_0
4_85506_1
7_85703_0


##New Split

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=1, shuffle=True)
IDX=[x for x in range(10)]
IDY=[x for x in range(10)]
for train_index, val_index in cv.split(metadat,metadat):
  print(train_index, val_index)

[ 0  1  2  4  5  7  8  9 11 12 13 14 15 17 18 19] [ 3  6 10 16]
[ 0  1  3  5  6  7  8  9 10 11 12 13 15 16 18 19] [ 2  4 14 17]
[ 2  3  4  5  6  8  9 10 11 12 14 15 16 17 18 19] [ 0  1  7 13]
[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 16 17] [ 9 15 18 19]
[ 0  1  2  3  4  6  7  9 10 13 14 15 16 17 18 19] [ 5  8 11 12]


In [ ]:
for i,x in enumerate(train_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_train=X[ri:ri+size]
    Y_train=Y[ri:ri+size]
  else:
    X_train=np.concatenate((X_train,X[ri:ri+size]),axis=0)
    Y_train=np.concatenate((Y_train,Y[ri:ri+size]),axis=0)
for i,x in enumerate(val_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_test=X[ri:ri+size]
    Y_test=Y[ri:ri+size]
  else:
    X_test=np.concatenate((X_test,X[ri:ri+size]),axis=0)
    Y_test=np.concatenate((Y_test,Y[ri:ri+size]),axis=0)

##LSH Attention as keras multihead subclass

In [ ]:
from tensorflow.python.ops.gen_math_ops import sqrt_eager_fallback
import collections
import math
import string
import numpy as np
import tensorflow as tf
from keras.layers.attention.multi_head_attention import MultiHeadAttention
from keras.utils import tf_utils




class LSHAttention(MultiHeadAttention):
  def __init__(self, *args, **kwargs):
    self.n_buckets= kwargs.pop('n_buckets', 4)
    super().__init__(*args, **kwargs)
  def _build_from_signature(self, query, value, key=None):
    with tf_utils.maybe_init_scope(self):
      self._hash=tf.keras.layers.EinsumDense("bjhd,dc->bjhc",output_shape=(None,None,self.n_buckets // 2))
      self.lsh_proj=tf.random.normal
    super()._build_from_signature(query, value, key=None)

  def call(
        self,
        query,
        value,
        key=None,
        attention_mask=None,
        return_attention_scores=False,
        training=None,
        use_causal_mask=False,
        ):
        attention_mask = self._compute_attention_mask(
            query,
            value,
            key=key,
            attention_mask=attention_mask,
            use_causal_mask=use_causal_mask,
        )

        if not self._built_from_signature:
            self._build_from_signature(query=query, value=value, key=key)
        if key is None:
            key = value

        query_is_ragged = isinstance(query, tf.RaggedTensor)
        if query_is_ragged:
            query_lengths = query.nested_row_lengths()
            query = query.to_tensor()

        key_is_ragged = isinstance(key, tf.RaggedTensor)
        value_is_ragged = isinstance(value, tf.RaggedTensor)
        if key_is_ragged and value_is_ragged:
            # Ensure they have the same shape.
            bounding_shape = tf.math.maximum(
                key.bounding_shape(), value.bounding_shape()
            )
            key = key.to_tensor(shape=bounding_shape)
            value = value.to_tensor(shape=bounding_shape)
        elif key_is_ragged:
            key = key.to_tensor(shape=tf.shape(value))
        elif value_is_ragged:
            value = value.to_tensor(shape=tf.shape(key))

        #   N = `num_attention_heads`
        #   H = `size_per_head`
        # `query` = [B, T, N ,H]
        query = self._query_dense(query)
        # `key` = [B, S, N, H]
        key = query
        # `value` = [B, S, N, H]
        value = self._value_dense(value)
        for i in range(value.shape[2]):
          auxAtt=tf.zeros_like(value[:,:,i,:])[:,:,None,:]
          #here we make the rotation matrix, that will hash the vectors
          random_rot=self.lsh_proj([value.shape[-1],self.n_buckets // 2])
          #hash=self._hash(value)

          #here we make the rotation using dot product, and making the buckets
          hash=tf.matmul(value[:,:,i,:],random_rot)
          hash=tf.concat([hash,-hash],axis=-1)

          #here we make each token be clasified to a bucket
          buckets=tf.math.argmax(hash,axis=-1)

          #This is the index that will sort the inputs, and is made by sorting the buckets
          index=tf.argsort(buckets,axis=-1)

          #here we sort the main inputs usinf the index
          sv=tf.gather(value[:,:,i,:],index,axis=1,batch_dims=-1)
          sq=sk=tf.gather(query[:,:,i,:],index,axis=1,batch_dims=-1)
          buckets=tf.gather(buckets,index,axis=1,batch_dims=-1)
          #And chunk the inputs based on how many buckets we have
          chunk_size=int(sv.shape[1]/self.n_buckets)

          for j in range(self.n_buckets):
            if j==0:
              v=sv[:,j*chunk_size:(j+2)*chunk_size,:]
              b=buckets[:,j*chunk_size:(j+2)*chunk_size]
              q=sq[:,j*chunk_size:(j+2)*chunk_size,:]
            elif j==self.n_buckets-1:
              v=sv[:,(j-1)*chunk_size:(j+1)*chunk_size,:]
              b=buckets[:,(j-1)*chunk_size:(j+1)*chunk_size]
              q=sq[:,(j-1)*chunk_size:(j+1)*chunk_size,:]
            else:
              v=sv[:,(j-1)*chunk_size:(j+2)*chunk_size,:]
              b=buckets[:,(j-1)*chunk_size:(j+2)*chunk_size]
              q=sq[:,(j-1)*chunk_size:(j+2)*chunk_size,:]
            mask = tf.cast(tf.equal(b, j), tf.float32)
            mask = tf.expand_dims(mask, axis=-1)
            mask = tf.repeat(mask, repeats=v.shape[-1], axis=-1)
            v=v*mask
            q=q*mask
            attention_output, attention_scores = self._compute_attention(
                q[:,:,None,:], q[:,:,None,:], v[:,:,None,:], attention_mask, training
            )
            if j==0 or j==1:
              newatt=tf.pad(attention_output,[[0,0],[0,chunk_size*(self.n_buckets-2-j)],[0,0], [0,0]])
              auxAtt=tf.add(auxAtt,newatt)
              del newatt
            elif j==self.n_buckets-1:
              newatt=tf.pad(attention_output,[[0,0],[chunk_size*(j-1),0],[0,0], [0,0]])
              auxAtt=tf.add(auxAtt,newatt)
            else:
              if j-1 != 0:
                newatt=tf.pad(attention_output,[[0,0],[chunk_size*(j-1),0],[0,0], [0,0]])
                if self.n_buckets-2-j !=0:
                  newatt=tf.pad(newatt,[[0,0],[0,chunk_size*(self.n_buckets-2-j)],[0,0], [0,0]])
              auxAtt=tf.add(auxAtt,newatt)
          attention_output=auxAtt
          if i==0:
            att_o=attention_output
          else:
            att_o=tf.concat([att_o,attention_output],axis=2)
        attention_output = att_o
        attention_output = self._output_dense(attention_output)
        if query_is_ragged:
            attention_output = tf.RaggedTensor.from_tensor(
                attention_output, lengths=query_lengths
        )
        if return_attention_scores:
            return attention_output, attention_scores
        return attention_output

##Patch wise

In [ ]:
from keras.layers import Layer
import keras.backend as K
import torch.nn as nn
import tensorflow as tf

In [ ]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou
def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

In [ ]:
def dice_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      #None,128,128,1
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      #None,128,128
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += dice_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def iou_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += iou_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def conv_block(X,f,d=0.1,group=1):
  c = tf.keras.layers.Conv2D(f[0], (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',groups=group)(X)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[1], (3, 3), kernel_initializer='he_normal', padding='same', groups=group)(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  s = tf.keras.layers.Conv2D(f[1], (1, 1), kernel_initializer='he_normal', padding='same')(X)
  s = tf.keras.layers.BatchNormalization(axis=3)(s)
  c = tf.keras.layers.Add()([c,s])
  c = tf.keras.layers.ReLU()(c)
  return c,c

In [ ]:
import sys

In [ ]:
def Flatten(M):
  s1=M.shape[1]
  s2=M.shape[2]
  s3=M.shape[3]
  flat=s1*s2
  M= tf.reshape(M,[-1,flat,s3])
  return M,s1,s2,s3

In [ ]:
def Flatten_LSH(M,N,nheads,dimk,k=128):
  M,s1,s2,s3= Flatten(M)
  N,_,_,_= Flatten(N)
  pm=LSHAttention(num_heads=nheads,key_dim=dimk,dropout=0.2,n_buckets=k)(M,M)
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
def vit(X,Y,n_heads=16,psize=16,k=256,dim="4D"):
  npatch=int(X.shape[1]/psize)
  psizey=int(psize/2)
  kdim=int(X.shape[3]/n_heads)
  Y=tf.keras.layers.Dense(X.shape[3])(Y)
  if npatch==0:
    pm=Flatten_LSH(X,Y,n_heads,kdim,k)
    return pm
  for i in range(npatch):
    st=i*psize
    sty=i*psizey
    end=st+psize
    endy=sty+psizey
    for j in range(npatch):
      st2=j*psize
      end2=st2+psize
      sty2=j*psizey
      endy2=sty2+psizey
      patch= X[:,st:end,st2:end2,:]
      patch2= Y[:,sty:endy,sty2:endy2,:]
      pm=Flatten_LSH(patch,patch2,n_heads,kdim,k)
      if j==0:
        A=pm
      else:
        A=tf.concat([A,pm],2)
    if i==0:
      V=A
    else:
      V=tf.concat([V,A],1)
  return V



### Vit

In [ ]:
import tensorflow as tf

def create_model():
  nheads=8
  sizep=16
  gr=4
  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path
  m1= vit(z4,z5,nheads,sizep,k=4)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= vit(z3,z6,nheads,sizep,k=4)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= vit(z2,z7,nheads,sizep,k=4)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= vit(z1,z8,nheads,sizep,k=4)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###LSH without Patches

In [ ]:
def Flatten(M):
  s1=M.shape[1]
  s2=M.shape[2]
  s3=M.shape[3]
  flat=s1*s2
  M= tf.reshape(M,[-1,flat,s3])
  return M,s1,s2,s3

In [ ]:
def LSH_lay(M,nheads,k=128,nbuckets=4):
  M,s1,s2,s3= Flatten(M)
  dimk=int(M.shape[2]/nheads)
  pm=LSHAttention(num_heads=nheads,key_dim=dimk,dropout=0.2,n_buckets=nbuckets)(M,M)
  pm=tf.reshape(pm,[-1,s1,s2,s3])
  return pm

In [ ]:
import tensorflow as tf

def create_model():
  nheads=4
  sizep=32
  gr=4

  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path

  m1= LSH_lay(z4,nheads)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= LSH_lay(z3,nheads,nbuckets=8)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= LSH_lay(z2,nheads,nbuckets=8)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= LSH_lay(z1,nheads,nbuckets=32)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###Test

In [ ]:
model=create_model()
model.fit(X_train,Y_train,batch_size=16,epochs=25)

Epoch 1/25


for split version, time for execution 42 m

patched version time for execution 34 m

In [ ]:
scores= model.evaluate(X_test, Y_test, verbose=0)
print(f'Score for fold {0}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')

Score for fold 0: loss of 0.036873139441013336; accuracy of 98.70224595069885% iou_coef_multilabel of 68.26137900352478% dice_coef_multilabel of 72.85516262054443%


Patch size=32 k=32:

Score for fold 0: loss of 0.02660856954753399; accuracy of 99.21053647994995% iou_coef_multilabel of 73.97453188896179% dice_coef_multilabel of 77.59955525398254%

No Patch k=128:

Score for fold 0: loss of 0.03675605729222298; accuracy of 98.8860547542572% iou_coef_multilabel of 69.78949904441833% dice_coef_multilabel of 74.2724359035492%

Patch size=32 k=256:
Score for fold 0: loss of 0.11757280677556992; accuracy of 96.79576754570007% iou_coef_multilabel of 71.6090202331543% dice_coef_multilabel of 79.81376647949219%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

####Patch test

k=32

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.029409149661660194; accuracy of 98.91234040260315% iou_coef_multilabel of 70.78478932380676% dice_coef_multilabel of 75.16862750053406%
Score for fold 2: loss of 0.042917650192976; accuracy of 98.51773381233215% iou_coef_multilabel of 70.23463249206543% dice_coef_multilabel of 75.7128119468689%
Score for fold 3: loss of 0.04444853588938713; accuracy of 98.51000308990479% iou_coef_multilabel of 67.47883558273315% dice_coef_multilabel of 72.66407012939453%
Score for fold 4: loss of 0.10042478889226913; accuracy of 96.02788090705872% iou_coef_multilabel of 55.91818690299988% dice_coef_multilabel of 63.355553150177%
Score for fold 5: loss of 0.03272783383727074; accuracy of 98.82147312164307% iou_coef_multilabel of 69.75324749946594% dice_coef_multilabel of 74.68957304954529%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.025224272161722183; accuracy of 99.0780770778656% iou_coef_multilabel of 75.7142961025238% dice_coef_multilabel of 79.58106994628906%
Score for fold 2: loss of 0.04713413491845131; accuracy of 98.5433280467987% iou_coef_multilabel of 74.99583959579468% dice_coef_multilabel of 80.19270896911621%
Score for fold 3: loss of 0.04239453375339508; accuracy of 98.56003522872925% iou_coef_multilabel of 71.22631072998047% dice_coef_multilabel of 76.35141015052795%
Score for fold 4: loss of 0.04546702280640602; accuracy of 98.31485748291016% iou_coef_multilabel of 70.64926028251648% dice_coef_multilabel of 77.11712121963501%
Score for fold 5: loss of 0.032865818589925766; accuracy of 98.83249998092651% iou_coef_multilabel of 71.10702991485596% dice_coef_multilabel of 75.85053443908691%


k=256

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.016601325944066048; accuracy of 99.49246644973755% iou_coef_multilabel of 80.1440417766571% dice_coef_multilabel of 82.72783160209656%
Score for fold 2: loss of 0.02792888507246971; accuracy of 99.11463856697083% iou_coef_multilabel of 77.49691009521484% dice_coef_multilabel of 81.86010122299194%
Score for fold 3: loss of 0.033090461045503616; accuracy of 99.0357756614685% iou_coef_multilabel of 75.54389238357544% dice_coef_multilabel of 79.85571622848511%
Score for fold 4: loss of 0.02735254541039467; accuracy of 99.02375340461731% iou_coef_multilabel of 75.38833618164062% dice_coef_multilabel of 80.59991002082825%
Score for fold 5: loss of 0.02401789091527462; accuracy of 99.26612973213196% iou_coef_multilabel of 76.66497826576233% dice_coef_multilabel of 80.13312816619873%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01717947982251644; accuracy of 99.4581937789917% iou_coef_multilabel of 79.65168952941895% dice_coef_multilabel of 82.61964917182922%
Score for fold 2: loss of 0.03299105167388916; accuracy of 99.06346797943115% iou_coef_multilabel of 78.3797025680542% dice_coef_multilabel of 82.73652791976929%
Score for fold 3: loss of 0.034237224608659744; accuracy of 99.15516972541809% iou_coef_multilabel of 78.44327688217163% dice_coef_multilabel of 82.67260193824768%


KeyboardInterrupt: ignored

####No Patch test

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.02731938473880291; accuracy of 99.00254607200623% iou_coef_multilabel of 71.55993580818176% dice_coef_multilabel of 75.51640272140503%
Score for fold 2: loss of 0.04416176304221153; accuracy of 98.46460819244385% iou_coef_multilabel of 70.76845169067383% dice_coef_multilabel of 76.32638216018677%
Score for fold 3: loss of 0.04237723350524902; accuracy of 98.45744967460632% iou_coef_multilabel of 67.4326241016388% dice_coef_multilabel of 72.71857857704163%
Score for fold 4: loss of 0.04793655872344971; accuracy of 98.23458194732666% iou_coef_multilabel of 68.04465651512146% dice_coef_multilabel of 74.47894811630249%
Score for fold 5: loss of 0.0348425917327404; accuracy of 98.7302839756012% iou_coef_multilabel of 65.84467887878418% dice_coef_multilabel of 70.64203023910522%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

KeyboardInterrupt: ignored